In [3]:
from bs4 import BeautifulSoup     #网页解析，获取数据
import re       #正则表达式，进行文字匹配
import urllib.request,urllib.error      #制定URL，获取网页数据

import sqlite3  #进行SQLite数据库操作
import pandas as pd #存储数据及处理数据
import numpy as np
import requests
import random
import time
from collections import defaultdict
from collections import deque
from collections import Counter
import collections
import json
import sys


In [2]:
from collections import Counter
import pandas as pd  # 存储数据及处理数据
import numpy as np
import json


# 根据产品编码找到该产品t步以内的所有连接节点
def find_connect_nodes(code, t, df_net):
    # code是节点编码，str
    # df_nodes是节点表，dataframe
    # df_net是联系表，dataframe
    code_ls = []
    code_ls.append([code])

    flag = 1
    step = 1
    ls_forecast = []
    while step <= t:
        _exist_list = []
        for k in range(len(code_ls)):
            _exist_list.extend(code_ls[k])
        _possible_list = []

        for i in range(len(code_ls[-1])):
            _industryCO = code_ls[-1][i]
            df_temp = df_net.loc[(df_net['source'] == _industryCO) | (df_net['target'] == _industryCO)]
            if len(df_temp) > 0:
                _industryCOs_1 = df_temp.loc[:, 'source'].unique()
                _industryCOs_2 = df_temp.loc[:, 'target'].unique()
                for m in range(len(_industryCOs_1)):
                    _industryCO_ = _industryCOs_1[m]
                    if _industryCO_ not in _exist_list:
                        _possible_list.append(_industryCO_)
                        ls_forecast.append(_industryCO_)

                for m in range(len(_industryCOs_2)):
                    _industryCO_ = _industryCOs_2[m]
                    if _industryCO_ not in _exist_list:
                        _possible_list.append(_industryCO_)
                        ls_forecast.append(_industryCO_)

        _possible_list = list(set(_possible_list))
        code_ls.append(_possible_list)
        step += 1
    return list(set(ls_forecast))


# 根据产品编码找到该产品所属的产业链
def code_to_chain(code, dataframe):
    # code是节点编码,str
    # dataframe是节点表E:\yulin\connection\gephi_total\yulin_total_nodes_temp.csv,dataframe
    df_temp = dataframe.loc[dataframe['Id'] == code]
    chain_temp = df_temp['chain'].values.tolist()[0]
    ls = json.loads(chain_temp.replace("'", '"'))
    return ls


# 根据产品编码找到该产品的名称
def code_to_name(code, dataframe):
    # code是节点编码,str
    # dataframe是节点表E:\yulin\connection\gephi_total\yulin_total_nodes_temp.csv,dataframe
    df_temp = dataframe.loc[dataframe['Id'] == code]
    name_ = df_temp['Label'].tolist()[0]
    return name_


# 找到产业链对应的产品编码列表
# dataframe是节点表E:\yulin\connection\gephi_total\yulin_total_nodes_temp.csv
def find_chain_ls(chain_name_ls, dataframe):
    # chain_name_ls是要找的产业链名称列表,list
    # dataframe是
    ls = []
    for i in chain_name_ls:
        df_temp = dataframe.loc[dataframe['chain'].str.contains(i)]
        ls.extend(df_temp['Id'].tolist())
    return ls


# 根据产业链的组成产品，找到该产业链的联系数据行
def find_chain_connection(chain_nodes, df_chain_net):
    # chain_nodes是该产业链的下辖产品列表，list
    # df_chain_net是产业链的联系列表，dataframe
    df_chain_temp = df_chain_net.loc[
        (df_chain_net['source'].isin(chain_nodes)) & (df_chain_net['target'].isin(chain_nodes))]
    df_chain_temp = df_chain_temp.loc[:, ['source', 'target', 'type', 'weight', 'direction']]
    return df_chain_temp


# 根据产品名称找到对应的产品编码
def name_to_code(name_ls, dataframe):
    # name_ls是要找的产品名称列表，list
    # dataframe是入库表
    for a in name_ls:
        location = np.where(dataframe == a)
        columns_ls = dataframe.columns.tolist()

        code_ls = []
        for i in range(len(location[0])):
            index = location[0][i]
            column = location[1][i]
            if column < 8:
                continue
            else:
                tier = columns_ls[column][-1]
                col_name = 'code_' + tier
                code_ls.append(dataframe.iloc[index][col_name])

    return list(Counter(code_ls))


# 预测模拟产品的上下游补充发展方向
def find_potential_nodes(ls_adv, ls_forbidden, wight_value, df_product_net):
    # ls_adv是本地优势产品，list
    # ls_local是本地所有产品列表，list
    # forbidden是预先设置的排除产业，list
    # wight_value是设定的阈值，float
    # df_product_net是产品上下游联系网络，dataframe

    _district_industryCOs = ls_adv
    ls_forbidden = ls_forbidden
    _simulate_list = [_district_industryCOs]
    # 从一步开始向多步扩展
    flag = 1
    t = 1
    ls_forcast = []
    while flag == 1:
        _exist_list = []
        # 把所有优势产品输入_exist_list
        for k in range(0, len(_simulate_list)):
            _exist_list.extend(_simulate_list[k])
        c = 0
        _possible_list = []

        for j in range(len(_simulate_list[-1])):
            _industryCO = _simulate_list[-1][j]
            if _industryCO not in ls_forbidden:
                # 判断该优势产品属于本地产品，且不属于本地排除产品，然后从该优势产品开始向外找，一步联系值大于等于wight_value的节点
                _df_industryCO_proximity = df_product_net.loc[((df_product_net['source'] == _industryCO) |
                                                               (df_product_net['target'] == _industryCO)) &
                                                              (df_product_net['weight'] >= wight_value)]
                if len(_df_industryCO_proximity) > 0:
                    # 取出该一步联系所能联系到的所有产品节点
                    _industryCOs_1 = _df_industryCO_proximity.loc[:, 'source'].unique()
                    _industryCOs_2 = _df_industryCO_proximity.loc[:, 'target'].unique()

                    for m in range(len(_industryCOs_1)):
                        _industryCO_ = _industryCOs_1[m]
                        # 如果该产品节点不是已经找过的产品（作为起点的产品），也不属于本地排除产品，则加入到_possible_list
                        if _industryCO_ not in _exist_list and _industryCO not in ls_forbidden:
                            _possible_list.append(_industryCO_)
                            ls_a = []
                            ls_a.append(t)
                            ls_a.append(_industryCO)
                            ls_a.append(_industryCO_)

                            ls_forcast.append(ls_a)
                            c = c + 1

                    for m in range(len(_industryCOs_2)):
                        _industryCO_ = _industryCOs_2[m]
                        # 如果该产品节点不是已经找过的产品（作为起点的产品），则加入到_possible_list
                        if _industryCO_ not in _exist_list and _industryCO not in ls_forbidden:
                            _possible_list.append(_industryCO_)
                            ls_a = []
                            ls_a.append(t)
                            ls_a.append(_industryCO)
                            ls_a.append(_industryCO_)
                            ls_forcast.append(ls_a)
                            c = c + 1
        if c == 0:
            # 当c等于0时，说明一步联系找到的都是已有产品，无法再继续找了
            flag = 0
        else:
            # 当c不等于0时，说明还可以继续往下找
            flag = 1
            t = t + 1

            _possible_list = list(set(_possible_list))
            _simulate_list.append(_possible_list)
    return ls_forcast


# 根据模拟节点得到这些节点的邻接矩阵
# 返回的邻接矩阵是一个列表，可以用np.array()来转变为矩阵
def adjacence_matrix(code_ls, dataframe):
    # code_ls是产业链已有产品节点编码和预测出的节点编码一起组成的节点编码表
    # dataframe是节点间联系表
    df_temp = dataframe.loc[(dataframe['source'].isin(code_ls)) & (dataframe['target'].isin(code_ls))]

    # 根据code_ls生成各节点编码的标识字典
    member_dict = {}
    name_ = 0
    for i in code_ls:
        member_dict[i] = name_
        name_ += 1

    # 节点个数为len(code_ls)，基础矩阵为g_
    n = len(code_ls)
    g_ = [[0 for i in range(n)] for i in range(n)]

    # 根据节点间的联系数据将邻接矩阵填入数据
    df_temp_1 = df_temp.loc[:, ['source', 'target']]
    df_temp_1_ls = df_temp_1.values.tolist()
    for i in df_temp_1_ls:
        x_index = member_dict[i[0]]
        y_index = member_dict[i[1]]

        g_[x_index][y_index] = 1
        g_[y_index][x_index] = 1

    return g_, member_dict


# 根据邻接矩阵求得可达矩阵
def get_reach_matrix(matrix):
    # matrix是邻接矩阵

    # 输入邻接矩阵和相同长度的单位矩阵
    adj_matrix = np.mat(matrix)
    identity_matrix = np.identity(len(adj_matrix))
    # 第k+1步更新的矩阵
    new_matrix_ = adj_matrix + identity_matrix
    # 第k步更新的矩阵
    old_matrix_ = new_matrix_

    flag = 0
    step = 1
    while flag == 0:
        old_matrix_ = new_matrix_
        new_matrix_ = old_matrix_ * (adj_matrix + identity_matrix)
        for i in range(len(new_matrix_)):
            for j in range(len(new_matrix_)):
                if new_matrix_[i, j] >= 1:
                    new_matrix_[i, j] = 1
        step += 1
        if (old_matrix_ == new_matrix_).all():
            flag = 1

    return new_matrix_


# 最短路径法
# 广度优先搜索，一层一层往下搜索，直到找到终点为止，耗时有点长
def find_shortest_path_(start_node, end_node, all_nodes, ls_forbidden, df_net):
    # start_node是起点产品节点的编码，str
    # end_node是终点产品节点的编码，str
    # all_nodes是图谱里的所有产品节点，包含起点和终点，list
    # ls_forbidden是设定好的排除产品，不会出现在模拟结果中
    # df_net是全国产品节点间联系数据，dataframe

    # 初始化起点集合
    s_1 = [[start_node]]
    s_2 = [start_node]
    # 初始化剩余点集合
    u = [x for x in all_nodes if x not in s_2]
    t = 1
    ls_forecast = []

    while end_node in u:
        ls_nodes_temp_ = []
        for i in s_1[-1]:
            df_temp_ = df_net.loc[(df_net['source'] == i) | (df_net['target'] == i)]
            if len(df_temp_) > 0:
                ls_temp_ = df_temp_.loc[:, ['source', 'target']].values.tolist()
                ls_temp_ = [a for x in ls_temp_ for a in x]
                ls_temp_ = list(Counter(ls_temp_))
                ls_temp_ = [x for x in ls_temp_ if x not in s_2 and x not in ls_forbidden]
                ls_temp_ = list(Counter(ls_temp_))

                for j in ls_temp_:
                    ls_forecast_i = []
                    df_weight = df_temp_.loc[(df_temp_['source'] == j) | (df_temp_['target'] == j)]
                    weight = np.mean(df_weight['weight'].tolist())
                    ls_forecast_i.extend([t, i, j, weight])
                    ls_forecast.append(ls_forecast_i)
                ls_nodes_temp_.extend(ls_temp_)
            else:
                continue
        if len(ls_nodes_temp_) == 0:
            return [], []
        else:
            s_1.append(ls_nodes_temp_)
            s_2.extend(ls_nodes_temp_)
            t += 1
            u = [x for x in all_nodes if x not in s_2]
            # print(t, ls_nodes_temp_)
    # df_forecast 是最少步数可以从起点走到终点的数据集合
    df_forecast = pd.DataFrame(ls_forecast, columns=['t', 'origin', 'destination', 'weight'])
    location_end = np.where(df_forecast == end_node)
    if len(location_end[0]) != 0:
        # 根据路径数据，还原从起点到终点的路径path
        max_t = max(df_forecast['t'].tolist())
        t_temp = max_t
        path = [[end_node]]
        while t_temp >= 1:
            path_a = []
            for i in path:
                df_temp = df_forecast.loc[(df_forecast['t'] == t_temp) & (df_forecast['destination'] == i[0])]
                df_temp_ls = df_temp['origin'].tolist()

                for a in df_temp_ls:
                    ls_a = [a]
                    ls_a.extend(i)
                    path_a.append(ls_a)

            path = path_a
            t_temp -= 1

        # 根据path列表求得每条路径对应的代价之和
        path_weight = []
        for i in path:
            weight_i = []
            for j in range(len(i) - 1):
                df_weight = df_forecast.loc[(df_forecast['origin'] == i[j]) & (df_forecast['destination'] == i[j + 1])]
                weight_ij = df_weight['weight'].tolist()[0]
                weight_i.append(1 / weight_ij)
            path_weight.append(sum(weight_i))
        # 根据path_weight列表找到代价最小的那条路径
        path_weight_min = min(path_weight)
        path_min = [path[x] for x in range(len(path)) if path_weight[x] == path_weight_min]
        return path_min, path_weight_min
    else:
        return [], []


# 根据模拟过程文件，找到对应模拟结果形成的完整链路
def full_path_(dataframe, end_node):
    location = np.where(dataframe == end_node)
    location_end = []
    for i in range(len(location[0])):
        if location[1][i] == 2:
            location_i = [location[0][i],location[1][i]]
            location_end.append(location_i)

    if len(location_end) == 0:
        print('wrong dataframe, change it')
    else:
        index = location_end[-1][0]

    df_forcast = dataframe.loc[:index,:]
    max_t = max(df_forcast['t'].tolist())
    t_temp = max_t
    path = []
    path.append([end_node])
    while t_temp >= 1:
        path_a = []
        for i in path:
            df_temp = df_forcast.loc[(df_forcast['t'] == t_temp) & (df_forcast['destination'] == i[0])]
            df_temp_ls = df_temp['origin'].tolist()

            for a in df_temp_ls:
                ls_a = []
                ls_a.append(a)
                ls_a.extend(i)
                path_a.append(ls_a)

        path = path_a
        t_temp -= 1
    return path


# 根据路径计算weight之和，作为代价计算的函数
def path_weight_count(path_ls, df_net):
    # path_ls是路径列表
    # df_net是联系表，dataframe
    weight_ls_temp_ = []
    for i in range(len(path_ls)):
        if i < (len(path_ls) - 1):
            ls_i = []
            ls_i.append(path_ls[i])
            ls_i.append(path_ls[i + 1])
            df_temp = df_net.loc[(df_net['source'].isin(ls_i)) & (df_net['target'].isin(ls_i))]
            weight = df_temp['weight'].tolist()[0]
            if weight != 0:
                weight = 1 / weight
                weight_ls_temp_.append(weight)
            else:
                return 9999

    return sum(weight_ls_temp_)


# 最小代价法
# 注：按照dijkstra算法，需要遍历图谱中的每一个节点，对算法的时间效率影响较大
def find_minimal_path_(start_node, end_node, all_nodes, ls_forbidden, df_net):
    # start_node是起点产品节点的编码，str
    # end_node是终点产品节点的编码，str
    # all_nodes是图谱里的所有产品节点，包含起点和终点，list
    # df_net是全国产品节点间联系数据，dataframe

    # 初始化起点集合
    s_1 = [[start_node]]
    s_2 = [start_node]
    # 初始化剩余点集合
    u = [x for x in all_nodes if x not in s_2]
    # 初始化最短距离数据集合
    dist_ = dict(zip(all_nodes, [9999] * len(all_nodes)))
    dist_[start_node] = 0
    # 初始化路径集合
    path_ = dict(zip(all_nodes, [[]] * len(all_nodes)))

    while end_node in u:
        for a in s_1[-1]:
            df_temp_ = df_net.loc[(df_net['source'] == a) | (df_net['target'] == a)]
            ls_temp_ = df_temp_.loc[:, ['source', 'target']].values.tolist()
            ls_temp_ = [x_1 for x in ls_temp_ for x_1 in x]
            ls_temp_ = list(Counter(ls_temp_))
            ls_temp_ = [x for x in ls_temp_ if x not in s_2]
            for i in ls_temp_:

                df_temp_i_ = df_temp_.loc[(df_temp_['source'] == i) | (df_temp_['target'] == i)]
                weight_ = np.mean(df_temp_i_['weight'].tolist()[0])

                # if len(df_temp_i_['weight'].tolist()) > 1:
                #     print('有重复的联系数据', s_1[-1], i)
                weight_ = 1 / weight_ + dist_[a]
                if weight_ < dist_[i]:
                    dist_[i] = weight_
                    ls = [a]
                    path_[i] = ls
                elif weight_ == dist_[i]:
                    dist_[i] = weight_
                    ls = path_[i]
                    ls.append(a)
                    path_[i] = ls

                    # 找最小的weight值所对应的点，加入到s列表中
        min_dict = dict([(key, dist_[key]) for key in u])
        min_value_ = min(min_dict.values())
        ls_nodes_temp_ = [key for key, value in min_dict.items() if value == min_value_]
        # 将距离最短的起点重新添加进s_1和s_2，刷新u列表，进入下一个循环，知道找到终点
        s_1.append(ls_nodes_temp_)
        s_2.extend(ls_nodes_temp_)
        u = [x for x in all_nodes if x not in s_2]

    mini_path = [[end_node]]
    start = [path_[end_node]]
    start_path = [[path_[end_node]]]

    while start_node not in start[-1]:
        mini_path_ = []
        for b in range(len(start_path[-1])):
            for c in start_path[-1][b]:
                ls_b = [c]
                ls_b.extend(mini_path[b])
                mini_path_.append(ls_b)

        # mini_path是得到最短路径列表
        mini_path = mini_path_
        start_1 = []
        start_2 = []
        for d in mini_path:
            start_1.extend(path_[d[0]])
            start_2.append(path_[d[0]])
        start.append(start_1)
        start_path.append(start_2)

    # 当最短路径只有一条时，直接返回该路径和对应的weight值之和
    # mini_path的格式是两层列表[[]]
    if len(mini_path) == 1:
        for s in mini_path:
            s.insert(0, start_node)
        return mini_path, dist_[end_node]

    else:
        # 当weight值之和相同的路径有多条时，选择len步数长度最短的那一条
        shortest_len = 99
        shortest_index = [99]
        for s in range(len(mini_path)):

            len_i = len(mini_path[s])
            if len_i < shortest_len:
                shortest_index = [s]
                shortest_len = len_i
            elif len_i == shortest_len:
                shortest_index.append(s)
                shortest_len = len_i

        shortest_path = [mini_path[x] for x in shortest_index]
        shortest_path_result = []
        for t in shortest_path:
            forbidden_nodes = [x for x in t if x in ls_forbidden]
            if len(forbidden_nodes) == 0:
                t.insert(0, start_node)
                shortest_path_result.append(t)
        # 如果有多个相同步数，相同weight值的路径，则都返回结果，进行模拟结果补充
        # 返回相同步数，相同weight值的路径和对应的weight值之和
        return shortest_path_result, dist_[end_node]

In [4]:
# 主程序开始
# from prediction_functions import *
from collections import Counter
import pandas as pd

# 读取基底表和入库表
df_jidi = pd.read_csv('E:\\project_data\\04yulin\\connection\\prediction\\data\\yulin_fundamental_data_7.csv',encoding='gbk',dtype='str')
df_dataset = pd.read_csv('E:\\project_data\\04yulin\\connection\\prediction\\data\\yulin_dataset.csv',encoding='gbk',dtype='str')

# 读取联系表E:\yulin\connection\gephi_total\yulin_total_connection.csv
# df_net = pd.read_excel('E:\\yulin\\connection\\prediction\\data\\yulin_total_connection_new.xlsx',dtype={'weight':'float','source':'str','target':'str'})
# df_chain_net = pd.read_csv('E:\\yulin\\connection\\prediction\\data\\yulin_chain_connection_code.csv',encoding='gbk',dtype={'weight':'float','source':'str','target':'str'})
df_net = pd.read_csv('E:\\project_data\\04yulin\\connection\\prediction\\data\\yulin_total_connection_.csv',encoding='gbk', dtype={'weight':'float','source':'str','target':'str'})
df_chain_net = pd.read_csv('E:\\project_data\\04yulin\\connection\\prediction\\data\\yulin_chain_connection_code.csv',encoding='gbk',dtype={'weight':'float','source':'str','target':'str'})

# 读取节点表E:\yulin\connection\gephi_total\yulin_total_nodes_temp.csv
# local=1为本地产品；local=0为非本地产品
# chain不为0的说明产品属于某条产业链
# df_nodes = pd.read_csv('E:\\yulin\\connection\\prediction\\data\\yulin_total_nodes_temp.csv',encoding='gbk',dtype='str')
df_nodes = pd.read_csv('E:\\project_data\\04yulin\\connection\\prediction\\data\\yulin_total_nodes_.csv',encoding='gbk',dtype='str')

# 快速模拟的优势产业为节点表里，local=1的产品，获取其编码id列表ls_local_best
df_local_best = df_nodes.loc[df_nodes['local']=='1']
ls_local_best = df_local_best['Id'].tolist()

all_nodes = df_nodes['Id'].tolist()

# 读取共现数据,df_co_ex_1是由空间溢出文件得到的共现关系，df_co_ex_2是没有空间溢出，直接求条件概率得到的共现关系
# df_co_ex_1 = pd.read_csv('E:\\yulin\\connection\\co_exist_tupu\\all_ind_weighted23f3_75_pp_2022_1_mst_thre_clear.csv', dtype={'Weight':'float','Source':'str','Target':'str'})
# df_co_ex_2 = pd.read_csv('E:\\yulin\\connection\\co_exist_tupu\\all_ind_weighted23f3_75_pp_2022_2_mst_thre_clear.csv', dtype={'Weight':'float','Source':'str','Target':'str'})

# 读取产品节点间的共现数据
df_nodes_co_ex_1 = pd.read_csv('E:\\project_data\\04yulin\\connection\\prediction\\data\\nodes_weighted23f3_75_pp_2022_1_mst_thre_clear.csv', dtype={'weight':'float','source':'str','target':'str'})

# 选择执行，使程序结果在编辑器中打印完整
pd.set_option('display.max_columns', 1000)
pd.set_option('display.width', 1000)
pd.set_option('display.max_colwidth', 1000)
pd.set_option("display.max_rows",1000)

In [82]:
len(all_nodes)

3556

In [16]:
# 主程序
# 输入模拟模式
type_ = input('请输入模拟模式（快速模拟/高级模拟）：')

# 设定联系强度对应值
ls_connection_weight = ['最强', '较强', '中等', '较弱', '最弱']
ls_connection_weight_chain = [4.5, 3.5, 2.5, 1.5, 0.5]
ls_connection_weight_co_ex = [0.8, 0.6, 0.4, 0.2, 0.1]

if type_ == '快速模拟':
    env_ = '无'  # 模拟场景在快速模拟中为空
else:
    # 输入模拟场景：补短板、锻长板、促转型、建集群、自定义
    env_ = input('请输入模拟场景(补短板/锻长板/促转型/建集群/自定义)：')  # 模拟场景在高级模拟中可自行选择

# 输入模拟对象产业链，注：这里输入的产业链名称字符串必须用双引号，exp：["轻纺产业链"]，可以输入多条产业链，用英文格式的逗号连接
# 注：输入产业链时，可以只输入产业链名称的简称，比如轻纺、光伏、大宗固废等。
# 注：（当用户是非链长和非管理员用户时，不会输入预测对象产业链，而是直接输入优势产品及种子产品）
chain = input('请输入产业链名称：')
ls_chain = json.loads(chain)

# 找到模拟对象产业链（可能不止一条产业链）中所有本地节点
df_local_chain = pd.DataFrame([])
for n in ls_chain:
    df_chain_local_n = df_nodes.loc[(df_nodes['chain'].str.contains(n)) & (df_nodes['local'] == '1')]
    df_local_chain = pd.concat([df_local_chain, df_chain_local_n], axis=0)

df_local_chain = df_local_chain.drop_duplicates(inplace=False)
ls_chain_local_nodes = df_local_chain['Id'].tolist()

# 根据模拟产业链，得到这些产业链由哪些节点组成
ls_chain_nodes = find_chain_ls(ls_chain, df_nodes)

if env_ == '无':
    # 快速模拟，条件配置：
    ls_adv = ls_chain_local_nodes  # 优势产业为输入的产业链所有本地产品，种子产业为无
    # 排除产品名称：原铝（电解铝），铝压延加工材,铝材,铝合金,粗纱机,毛纱,羊毛纱
    # 排除产品代码：'331603','3336','333601','33340105','362808','170202','17020201'
    ls_forbidden = ['331603', '3336', '3316', '333601', '33340105', '362808', '170202', '17020201']
    index_weight = ls_connection_weight.index('较强')
    weight_chain = ls_connection_weight_chain[index_weight]  # 上下游阈值大于等于3.5(较强)
    weight_co_ex = ls_connection_weight_co_ex[index_weight]  # 上下游阈值大于等于0.6(较强)

    search_step_ = 2  # 步数小于等于2步
    if_connection_ = '否'  # 是否全连通：********否*********
    path_calculation = '无'  # 算法名称：无
    # 备注：税务数据涉及的产品节点优先，产业链内部产品节点优先
elif env_ != '无':
    # 高级模拟，条件配置：
    # 输入优势产品，是否选择预置优势产品,预置种子产品为产业链中的本地既有产品
    if_preset_adv = input('是否选择预置的优势产品（是/否）：')
    if if_preset_adv == '是':
        ls_adv = ls_chain_local_nodes
    else:
        ls_adv = []
    # 用户完善优势产品列表，输入为列表形式，exp：["电力电子元器件","水泥混凝土制品"]，可以输入多个产品，用英文格式的逗号连接
    adv_nodes_plus = input('请输入要参与模拟的优势产品：')
    adv_nodes_plus_ls = json.loads(adv_nodes_plus)
    adv_nodes_minus = input('请输入不参与模拟的优势产品：')
    adv_nodes_minus_ls = json.loads(adv_nodes_minus)
    adv_nodes_plus_ls_code = []
    for x in adv_nodes_plus_ls:
        df_x = df_nodes.loc[df_nodes['Label'] == x]
        code_x = df_x['Id'].tolist()[0]
        adv_nodes_plus_ls_code.append(code_x)
    adv_nodes_minus_ls_code = []
    for x in adv_nodes_minus_ls:
        df_x = df_nodes.loc[df_nodes['Label'] == x]
        code_x = df_x['Id'].tolist()[0]
        adv_nodes_minus_ls_code.append(code_x)    
    ls_adv.extend(adv_nodes_plus_ls_code)
    ls_adv = list(set(ls_adv))
    ls_adv = [x for x in ls_adv if x not in adv_nodes_minus_ls_code]

    # 输入种子产品列表（选填内容），输入为列表形式，exp：["电力电子元器件","水泥混凝土制品"]，可以输入多个产品，用英文格式的逗号连接
    seed_nodes = input('请输入参与模拟的种子产品：')
    ls_seed = json.loads(seed_nodes)

    # 输入排除产品，是否选择预置的排除产品
    # 预置的排除产品名称：原铝（电解铝），铝，铝压延加工材,铝材,铝合金,粗纱机,毛纱,羊毛纱
    # 排除产品代码：'331603','3336','3316','333601','33340105','362808','170202','17020201'
    if_preset_forbid = input('是否使用预置的排除产品（是/否）：')
    if if_preset_forbid == '是':
        ls_forbidden = ['331603', '3336', '3316', '333601', '33340105', '362808', '170202', '17020201']
    else:
        ls_forbidden = []
    # 调整完善排除产品列表
    forbid_nodes_plus = input('请输入要增加的排除产品：')
    forbid_nodes_plus_ls = json.loads(forbid_nodes_plus)
    forbid_nodes_minus = input('请输入要减少的排除产品：')
    forbid_nodes_minus_ls = json.loads(forbid_nodes_minus)
    ls_forbidden.extend(forbid_nodes_plus_ls)
    ls_forbidden = list(set(ls_forbidden))
    ls_forbidden = [x for x in ls_forbidden if x not in forbid_nodes_minus_ls]  # 排除产品

    ls_adv = ls_adv + ls_seed
    ls_adv = list(set(ls_adv))

    if env_ == '补短板' or env_ == '促转型' or env_ == '自定义':
        # 输入模拟阈值:最强4、较强3、中等2、较弱1、最弱0
        weight = input('请输入参与模拟的联系强度（最强/较强/中等/较弱/最弱）：')
        index_weight = ls_connection_weight.index(weight)
        weight_chain = ls_connection_weight_chain[index_weight]
        weight_co_ex = ls_connection_weight_co_ex[index_weight]

        # 输入搜索半径：一到五步
        step_ = input('请输入模拟的搜索半径（一步/两步/三步/四步/五步）：')
        step_1 = ['一步', '两步', '三步', '四步', '五步']
        step_2 = [1, 2, 3, 4, 5]
        step_ind = step_1.index(step_)
        search_step_ = step_2[step_ind]
    elif env_ == '锻长板':
        # 是否自定阈值
        if_define_weight = input('是否使用默认搜索阈值（是/否）:')
        if if_define_weight == '否':
            # 当用户选择自定义阈值时，需要输入对应的阈值强度，参与模拟
            weight = input('请输入参与模拟的联系强度（最强/较强/中等/较弱/最弱）：')
            index_weight = ls_connection_weight.index(weight)
            weight_chain = ls_connection_weight_chain[index_weight]
            weight_co_ex = ls_connection_weight_co_ex[index_weight]
        elif if_define_weight == '是':
            # 当用户选择内置的阈值时，按照
            index_weight = ls_connection_weight.index('较强')
            weight_chain = ls_connection_weight_chain[index_weight]
            weight_co_ex = ls_connection_weight_co_ex[index_weight]
        # 输入搜索半径：一到五步
        step_ = input('请输入模拟的搜索半径（一步/两步/三步/四步/五步）：')
        step_1 = ['一步', '两步', '三步', '四步', '五步']
        step_2 = [1, 2, 3, 4, 5]
        step_ind = step_1.index(step_)
        search_step_ = step_2[step_ind]
    elif env_ == '建集群':
        # 是否自定阈值
        if_define_weight = input('是否使用默认搜索阈值（是/否）:')
        if if_define_weight == '否':
            # 当用户选择自定义阈值时，需要输入对应的阈值强度，参与模拟
            weight = input('请输入参与模拟的联系强度（最强/较强/中等/较弱/最弱）：')
            index_weight = ls_connection_weight.index(weight)
            weight_chain = ls_connection_weight_chain[index_weight]
            weight_co_ex = ls_connection_weight_co_ex[index_weight]
        elif if_define_weight == '是':
            # 当用户选择内置的阈值时，按照
            index_weight = ls_connection_weight.index('最强')
            weight_chain = ls_connection_weight_chain[index_weight]
            weight_co_ex = ls_connection_weight_co_ex[index_weight]

        # 是否自定阈值搜索半径
        if_define_step = input('是否使用默认搜索半径（是/否）:')
        if if_define_step == '否':

            step_ = input('请输入模拟的搜索半径（一步/两步/三步/四步/五步）：')
            step_1 = ['一步', '两步', '三步', '四步', '五步']
            step_2 = [1, 2, 3, 4, 5]
            step_ind = step_1.index(step_)
            search_step_ = step_2[step_ind]
        else:
            search_step_ = 1

# 根据以上数据，模拟出结果，并找到 weight 大于等于 weight_，且小于等于 search_step_ 步的预测模拟结果
ls_forecast_chain = find_potential_nodes(ls_adv, ls_forbidden, weight_chain, df_net)
ls_forecast_co_ex = find_potential_nodes(ls_adv, ls_forbidden, weight_co_ex, df_nodes_co_ex_1)
df_forecast_chain = pd.DataFrame(ls_forecast_chain, columns=['t', 'origin', 'destination'])
df_forecast_co_ex = pd.DataFrame(ls_forecast_co_ex, columns=['t', 'origin', 'destination'])
df_result_chain = df_forecast_chain.loc[df_forecast_chain['t'] <= search_step_]
df_result_co_ex = df_forecast_co_ex.loc[df_forecast_co_ex['t'] <= search_step_]
ls_pre_chain = df_result_chain['destination'].unique()
ls_pre_co_ex = df_result_co_ex['destination'].unique()
ls_pre = [x for x in ls_pre_chain if x in ls_pre_co_ex]  # 共现求得的模拟结果和上下游求得的模拟结果取交集
ls_pre = [x for x in ls_pre if x not in ls_local_best and x not in ls_forbidden]
ls_pre = list(set(ls_pre))

if env_ == '补短板' or env_ == '无':
    ls_final = [x for x in ls_pre if x in ls_chain_nodes]
elif env_ == '促转型':
    ls_final = [x for x in ls_pre if x not in ls_chain_nodes]
elif env_ == '自定义' or env_ == '建集群' or env_ == '锻长板':
    ls_final = [x for x in ls_pre]

# 根据模拟场景确定全联通和算法
if env_ == '补短板' or env_ == '促转型' or env_ == '自定义':
    # 输入是否全联通
    if_connection_ = input('请输入模拟结果是否全联通（是/否）：')
    # 当选择模拟结果全联通时，需要选择具体的全联通算法：最短路径（最少的步数）/最小代价（每一步都是关联最紧密的路径）
    if if_connection_ == '是':
        path_calculation = input('请输入算法选择（最短路径/最小代价）：')
    else:
        path_calculation = '无'
elif env_ == '锻长板':
    # 用户选择是否要按照默认情况进行全联通
    if_define_connection = input('是否使用默认全联通模拟参数（是/否）:')
    if if_define_connection == '否':
        if_connection_ = input('请输入模拟结果是否全联通（是/否）：')
    else:
        if_connection_ = '是'

    # 当选择模拟结果全联通时，需要选择具体的全联通算法：最短路径（最少的步数）/最小代价（每一步都是关联最紧密的路径）
    if if_connection_ == '是':
        path_calculation = input('请输入算法选择（最短路径/最小代价）：')
    else:
        path_calculation = '无'
elif env_ == '建集群':
    # 用户选择是否要按照默认情况进行全联通
    if_define_connection = input('是否使用默认全联通模拟参数（是/否）:')
    if if_define_connection == '否':
        if_connection_ = input('请输入模拟结果是否全联通（是/否）：')
    else:
        if_connection_ = '否'

    # 当选择模拟结果全联通时，需要选择具体的全联通算法：最短路径（最少的步数）/最小代价（每一步都是关联最紧密的路径）
    if if_connection_ == '是':
        path_calculation = input('请输入算法选择（最短路径/最小代价）：')
    else:
        path_calculation = '无'

# 根据是否全联通参数，确定是否需要进行全联通判断
if if_connection_ == '是':
    # 将模拟出的节点加入到之前设置的种子产品和优势产品中
    ls_chain_local_nodes_ = ls_adv + ls_final
    # 用得到的节点编码列表（模拟条件优势产品+种子产品+模拟结果产品）构成邻接矩阵，以此计算可达矩阵，判断这些点是否全连通
    adj_matrix_chain, member_dict_chain = adjacence_matrix(ls_chain_local_nodes_, df_net)
    adj_matrix_co_ex, member_dict_co_ex = adjacence_matrix(ls_chain_local_nodes_, df_nodes_co_ex_1)
    # print(adj_matrix)
    reach_matrix_chain = get_reach_matrix(adj_matrix_chain)  # 上下游到达矩阵
    reach_matrix_co_ex = get_reach_matrix(adj_matrix_co_ex)  # 共现到达矩阵

    # 当上下游或共现的可达矩阵中，有一个值为0的元素个数为0，则说明这些节点构成的网络已经达到全连通。
    if len(np.where(reach_matrix_chain == 0)[0]) == 0 or len(np.where(reach_matrix_co_ex == 0)[0]) == 0:
        print('模拟结果已构成全联通网络，直接输出结果。')

    else:
        ls_zero_chain = {}
        for i in range(len(reach_matrix_chain)):
            ls_i = reach_matrix_chain[i].tolist()[0]
            ls_zero_chain[i] = len([x for x in ls_i if x == 0])
        min_index_chain = []
        for key, item in ls_zero_chain.items():
            if item == min(ls_zero_chain.values()):
                min_index_chain.append(key)
            # print(key, item)

        ls_zero_co_ex = {}
        for i in range(len(reach_matrix_co_ex)):
            ls_i = reach_matrix_co_ex[i].tolist()[0]
            ls_zero_co_ex[i] = len([x for x in ls_i if x == 0])
        min_index_co_ex = []
        for key, item in ls_zero_co_ex.items():
            if item == min(ls_zero_co_ex.values()):
                min_index_co_ex.append(key)

        if min(ls_zero_co_ex.values()) < min(ls_zero_chain.values()):
            reach_matrix = reach_matrix_co_ex
            min_index_z = min_index_co_ex
            member_dict = member_dict_co_ex
        else:
            reach_matrix = reach_matrix_chain
            min_index_z = min_index_chain
            member_dict = member_dict_chain

        # 当预测出的产品与模拟条件产品并没有构成全连通时，需要通过算法寻找补充节点，find_shortest_path_算法慢的不行
        if path_calculation == '最短路径':
            # 首先找到没有形成全联通的点对有哪些
            location = np.where(reach_matrix == 0)
            path_total = []
            path_weight_total = []
            ls_add = []
            for i in range(len(location[0])):
                start_index = location[0][i]
                end_index = location[1][i]
                if end_index > start_index == min_index_z[0]:
                    start_node = list(member_dict.keys())[start_index]
                    end_node = list(member_dict.keys())[end_index]
                    # 用最短路径算法找到对应的补充路径
                    path, path_weight = find_shortest_path_(start_node, end_node, all_nodes, ls_forbidden, df_net)
                    path_total.extend(path)
                    path_weight_total.append(path_weight)

                    ls_nodes = [y for x in path_total for y in x]
                    ls_nodes = list(Counter(ls_nodes))
                    ls_add_i = [x for x in ls_nodes if x not in ls_adv and x not in ls_final]
                    ls_add.extend(ls_add_i)
                    ls_add = list(Counter(ls_add))
                    ls_nodes = ls_final + ls_adv + ls_add
                    adj_matrix, member_dict_i = adjacence_matrix(ls_nodes, df_net)
                    reach_matrix = get_reach_matrix(adj_matrix)
                    if len(np.where(reach_matrix == 0)[0]) == 0:
                        # 当已经构成全联通时跳出循环得到最终结果，新增的节点ls_add用于构成全联通
                        print('已通过补充节点获得全联通')
                        break
                    # else:
                    #     print(start_index, end_index)
                    #     print(start_node, end_node)
                    #     print(len(np.where(reach_matrix == 0)[0]))
                    #     print(ls_add)
            ls_add = list(Counter(ls_add))

        elif path_calculation == '最小代价':
            # 首先找到没有形成全联通的点对有哪些
            location = np.where(reach_matrix == 0)
            path_total = []
            path_weight_total = []
            ls_add = []
            for i in range(len(location[0])):
                start_index = location[0][i]
                end_index = location[1][i]
                if end_index > start_index == min_index_z[0]:
                    start_node = list(member_dict.keys())[start_index]
                    end_node = list(member_dict.keys())[end_index]
                    # 用最小代价算法寻找全联通补充节点
                    path, path_weight = find_minimal_path_(start_node, end_node, all_nodes, ls_forbidden, df_net)
                    path_total.extend(path)
                    path_weight_total.append(path_weight)

                    ls_nodes = [y for x in path_total for y in x]
                    ls_nodes = list(Counter(ls_nodes))
                    ls_add_i = [x for x in ls_nodes if x not in ls_adv and x not in ls_final]
                    ls_add.extend(ls_add_i)
                    ls_add = list(Counter(ls_add))
                    ls_nodes = ls_final + ls_adv + ls_add
                    adj_matrix, member_dict_i = adjacence_matrix(ls_nodes, df_net)
                    reach_matrix = get_reach_matrix(adj_matrix)
                    if len(np.where(reach_matrix == 0)[0]) == 0:
                        # 当已经构成全联通时跳出循环得到最终结果，新增的节点ls_add用于构成全联通
                        print('已通过补充节点获得全联通')
                        break
                    # else:
                    #     print(start_index, end_index)
                    #     print(start_node, end_node)
                    #     print(len(np.where(reach_matrix == 0)[0]))
                    #     print(ls_add)
            ls_add = list(Counter(ls_add))

        # 得到补充节点后，针对补充节点进行评估，与正式模拟出的节点评估分开
        ls_final_add = []
        for k in ls_add:
            ls_i = [k, code_to_name(k, df_nodes), code_to_chain(k, df_nodes)]
            t_k = 1
            # 然后找到该节点几步以内有多少本地节点
            ls_connect_k = find_connect_nodes(k, t_k, df_net)
            df_k_local = df_nodes.loc[(df_nodes['Id'].isin(ls_connect_k)) & (df_nodes['local'] == '1')]
            ls_i.append(len(df_k_local))

            # 看这些本地节点分别属于哪条产业链
            ls_chain_ = []
            ls_chain_k = df_k_local['chain'].tolist()
            for b in ls_chain_k:
                if len(b) > 1:
                    # print(b)
                    chain = json.loads(b.replace("'", '"'))
                    ls_chain_.extend(chain)
            ls_i.append(list(set(ls_chain_)))
            ls_final_add.append(ls_i)

        # 打印查看补充产品节点结果
        df_final_add = pd.DataFrame(ls_final_add, columns=['product_code',
                                                           'product_name', 'chain', 'local_support', 'loc_sup_chain'])
        df_final_add.sort_values(['local_support'], ascending=False, inplace=True)
        print('补充结果评价', len(df_final_add), df_final_add)

elif if_connection_ == '否' and path_calculation == '无':
    print('模拟结果不需要全联通，直接输出结果。')

# 当已经构成全连通网络，则不需要新增节点，与全联通参数选否的情况一样，直接对当前模拟出来的节点按照本地支持力度进行排序
# 根据预测到的节点编码，补充该节点属于哪条或哪几条产业链，以及其本地关联产品属于哪些产业链
ls_final_chain = []
for k in ls_final:
    ls_i = [k, code_to_name(k, df_nodes), code_to_chain(k, df_nodes)]
    t_k = 1
    # 然后找到该节点几步以内有多少本地节点
    ls_connect_k = find_connect_nodes(k, t_k, df_net)
    df_k_local = df_nodes.loc[(df_nodes['Id'].isin(ls_connect_k)) & (df_nodes['local'] == '1')]
    ls_i.append(len(df_k_local))

    # 看这些本地节点分别属于哪条产业链
    ls_chain_ = []
    ls_chain_k = df_k_local['chain'].tolist()
    for b in ls_chain_k:
        if len(b) > 1:
            # print(b)
            chain = json.loads(b.replace("'", '"'))
            ls_chain_.extend(chain)
    ls_i.append(list(set(ls_chain_)))
    ls_final_chain.append(ls_i)

# 打印查看模拟出的属于该产业链的产品节点结果
df_final = pd.DataFrame(ls_final_chain,
                        columns=['product_code', 'product_name', 'chain', 'local_support', 'loc_sup_chain'])
df_final.sort_values(['local_support'], ascending=False, inplace=True)
# df_final.reset_index()
# print('模拟结果评价', df_final.loc[:20, :])
print('模拟结果评价', len(df_final), df_final)

# 查询模拟结果的模拟链路
question = input('是否对模拟结果有疑问（是/否）：')
if question == '是':
    node_code = input('请输入要查询的产品编码（产品编码/全部）：')  # 产品编码可用["2342","3245"]的格式进行多选
    path_check = []
    path_check_name = []
    if node_code == '全部':
        for i in ls_final:
            path_i = full_path_(df_forecast_chain, i)
            path_check.extend(path_i)
            path_i_name = []
            for a in path_i:
                ls_b = []
                for b in a:
                    name_b = code_to_name(b, df_nodes)
                    ls_b.append(name_b)
                path_i_name.append(ls_b)
            path_check_name.extend(path_i_name)
    else:
        ls_end_nodes = json.loads(node_code)
        for i in ls_end_nodes:
            path_i = full_path_(df_forecast_chain, i)
            path_check.extend(path_i)
            path_i_name = []
            for a in path_i:
                ls_b = []
                for b in a:
                    name_b = code_to_name(b, df_nodes)
                    ls_b.append(name_b)
                path_i_name.append(ls_b)
            path_check_name.extend(path_i_name)
    path_result = []
    for i in range(len(path_check)):
        path_result_i = [path_check[i], path_check_name[i]]
        print(pd.DataFrame(path_result_i))
        path_result.append(path_result_i)
else:
    print('模拟结束')

请输入模拟模式（快速模拟/高级模拟）：高级模拟
请输入模拟场景(补短板/锻长板/促转型/建集群/自定义)：自定义
请输入产业链名称：["羊子"]
是否选择预置的优势产品（是/否）：否
请输入要参与模拟的优势产品：["羊"]
请输入不参与模拟的优势产品：[]
请输入参与模拟的种子产品：[]
是否使用预置的排除产品（是/否）：是
请输入要增加的排除产品：[]
请输入要减少的排除产品：[]
请输入参与模拟的联系强度（最强/较强/中等/较弱/最弱）：较强
请输入模拟的搜索半径（一步/两步/三步/四步/五步）：两步
请输入模拟结果是否全联通（是/否）：是
请输入算法选择（最短路径/最小代价）：最短路径
补充结果评价 0 Empty DataFrame
Columns: [product_code, product_name, chain, local_support, loc_sup_chain]
Index: []
模拟结果评价 2   product_code product_name    chain  local_support       loc_sup_chain
1     14110501        食品防腐剂        0             22  [羊子产业链, 黄河沿岸枣业产业链]
0     14040204       干酪（奶酪）  [羊子产业链]              5             [羊子产业链]
是否对模拟结果有疑问（是/否）：是
请输入要查询的产品编码（产品编码/全部）：全部
       0        1         2
0  30106  1440010  14040204
1      羊      乳制品    干酪（奶酪）
       0        1         2
0  30106  1440010  14110501
1      羊      乳制品     食品防腐剂


In [ ]:
ls_forecast_chain = find_potential_nodes(ls_adv, ls_forbidden, weight_chain, df_net)
ls_forecast_co_ex = find_potential_nodes(ls_adv, ls_forbidden, weight_co_ex, df_nodes_co_ex_1)
df_forecast_chain = pd.DataFrame(ls_forecast_chain, columns=['t', 'origin', 'destination'])
df_forecast_co_ex = pd.DataFrame(ls_forecast_co_ex, columns=['t', 'origin', 'destination'])
df_result_chain = df_forecast_chain.loc[df_forecast_chain['t'] <= search_step_]
df_result_co_ex = df_forecast_co_ex.loc[df_forecast_co_ex['t'] <= search_step_]
ls_pre_chain = df_result_chain['destination'].unique()
ls_pre_co_ex = df_result_co_ex['destination'].unique()
ls_pre = [x for x in ls_pre_chain if x in ls_pre_co_ex]  # 共现求得的模拟结果和上下游求得的模拟结果取交集
ls_pre = [x for x in ls_pre if x not in ls_local_best and x not in ls_forbidden]
ls_pre = list(set(ls_pre))

In [16]:
df_local_chain = pd.DataFrame([])
for n in ls_chain:
    df_chain_local_n = df_nodes.loc[(df_nodes['chain'].str.contains(n)) & (df_nodes['local'] == '1')]
    df_local_chain = pd.concat([df_local_chain, df_chain_local_n], axis=0)

df_local_chain = df_local_chain.drop_duplicates(inplace=False)
ls_chain_local_nodes = df_local_chain['Id'].tolist()

ls_chain_nodes = find_chain_ls(ls_chain, df_nodes)

if env_ != '无':
    # 高级模拟，条件配置：
    # 输入优势产品，是否选择预置优势产品,预置种子产品为产业链中的本地既有产品
    if_preset_adv = input('是否选择预置的优势产品（是/否）：')
    if if_preset_adv == '是':
        ls_adv = ls_chain_local_nodes
    else:
        ls_adv = []
    # 用户完善优势产品列表，输入为列表形式，exp：["电力电子元器件","水泥混凝土制品"]，可以输入多个产品，用英文格式的逗号连接
    adv_nodes_plus = input('请输入要参与模拟的优势产品：')
    adv_nodes_plus_ls = json.loads(adv_nodes_plus)
    adv_nodes_minus = input('请输入不参与模拟的优势产品：')
    adv_nodes_minus_ls = json.loads(adv_nodes_minus)
    adv_nodes_plus_ls_code = []
    for x in adv_nodes_plus_ls:
        df_x = df_nodes.loc[df_nodes['Label'] == x]
        code_x = df_x['Id'].tolist()[0]
        adv_nodes_plus_ls_code.append(code_x)
    adv_nodes_minus_ls_code = []
    for x in adv_nodes_minus_ls:
        df_x = df_nodes.loc[df_nodes['Label'] == x]
        code_x = df_x['Id'].tolist()[0]
        adv_nodes_minus_ls_code.append(code_x)    
    ls_adv.extend(adv_nodes_plus_ls_code)
    ls_adv = list(set(ls_adv))
    ls_adv = [x for x in ls_adv if x not in adv_nodes_minus_ls_code]

    # 输入种子产品列表（选填内容），输入为列表形式，exp：["电力电子元器件","水泥混凝土制品"]，可以输入多个产品，用英文格式的逗号连接
    seed_nodes = input('请输入参与模拟的种子产品：')
    ls_seed = json.loads(seed_nodes)

    # 输入排除产品，是否选择预置的排除产品
    # 预置的排除产品名称：原铝（电解铝），铝，铝压延加工材,铝材,铝合金,粗纱机,毛纱,羊毛纱
    # 排除产品代码：'331603','3336','3316','333601','33340105','362808','170202','17020201'
    if_preset_forbid = input('是否使用预置的排除产品（是/否）：')
    if if_preset_forbid == '是':
        ls_forbidden = ['331603', '3336', '3316', '333601', '33340105', '362808', '170202', '17020201']
    else:
        ls_forbidden = []
    # 调整完善排除产品列表
    forbid_nodes_plus = input('请输入要增加的排除产品：')
    forbid_nodes_plus_ls = json.loads(forbid_nodes_plus)
    forbid_nodes_minus = input('请输入要减少的排除产品：')
    forbid_nodes_minus_ls = json.loads(forbid_nodes_minus)
    ls_forbidden.extend(forbid_nodes_plus_ls)
    ls_forbidden = list(set(ls_forbidden))
    ls_forbidden = [x for x in ls_forbidden if x not in forbid_nodes_minus_ls]  # 排除产品

    ls_adv = ls_adv + ls_seed
    ls_adv = list(set(ls_adv))

    if env_ == '补短板' or env_ == '促转型' or env_ == '自定义':
        # 输入模拟阈值:最强4、较强3、中等2、较弱1、最弱0
        weight = input('请输入参与模拟的联系强度（最强/较强/中等/较弱/最弱）：')
        index_weight = ls_connection_weight.index(weight)
        weight_chain = ls_connection_weight_chain[index_weight]
        weight_co_ex = ls_connection_weight_co_ex[index_weight]

        # 输入搜索半径：一到五步
        step_ = input('请输入模拟的搜索半径（一步/两步/三步/四步/五步）：')
        step_1 = ['一步', '两步', '三步', '四步', '五步']
        step_2 = [1, 2, 3, 4, 5]
        step_ind = step_1.index(step_)
        search_step_ = step_2[step_ind]
    elif env_ == '锻长板':
        # 是否自定阈值
        if_define_weight = input('是否使用默认搜索阈值（是/否）:')
        if if_define_weight == '否':
            # 当用户选择自定义阈值时，需要输入对应的阈值强度，参与模拟
            weight = input('请输入参与模拟的联系强度（最强/较强/中等/较弱/最弱）：')
            index_weight = ls_connection_weight.index(weight)
            weight_chain = ls_connection_weight_chain[index_weight]
            weight_co_ex = ls_connection_weight_co_ex[index_weight]
        elif if_define_weight == '是':
            # 当用户选择内置的阈值时，按照
            index_weight = ls_connection_weight.index('较强')
            weight_chain = ls_connection_weight_chain[index_weight]
            weight_co_ex = ls_connection_weight_co_ex[index_weight]
        # 输入搜索半径：一到五步
        step_ = input('请输入模拟的搜索半径（一步/两步/三步/四步/五步）：')
        step_1 = ['一步', '两步', '三步', '四步', '五步']
        step_2 = [1, 2, 3, 4, 5]
        step_ind = step_1.index(step_)
        search_step_ = step_2[step_ind]
    elif env_ == '建集群':
        # 是否自定阈值
        if_define_weight = input('是否使用默认搜索阈值（是/否）:')
        if if_define_weight == '否':
            # 当用户选择自定义阈值时，需要输入对应的阈值强度，参与模拟
            weight = input('请输入参与模拟的联系强度（最强/较强/中等/较弱/最弱）：')
            index_weight = ls_connection_weight.index(weight)
            weight_chain = ls_connection_weight_chain[index_weight]
            weight_co_ex = ls_connection_weight_co_ex[index_weight]
        elif if_define_weight == '是':
            # 当用户选择内置的阈值时，按照
            index_weight = ls_connection_weight.index('最强')
            weight_chain = ls_connection_weight_chain[index_weight]
            weight_co_ex = ls_connection_weight_co_ex[index_weight]

        # 是否自定阈值搜索半径
        if_define_step = input('是否使用默认搜索半径（是/否）:')
        if if_define_step == '否':

            step_ = input('请输入模拟的搜索半径（一步/两步/三步/四步/五步）：')
            step_1 = ['一步', '两步', '三步', '四步', '五步']
            step_2 = [1, 2, 3, 4, 5]
            step_ind = step_1.index(step_)
            search_step_ = step_2[step_ind]
        else:
            search_step_ = 1

# 根据以上数据，模拟出结果，并找到 weight 大于等于 weight_，且小于等于 search_step_ 步的预测模拟结果
ls_forecast_chain = find_potential_nodes(ls_adv, ls_forbidden, weight_chain, df_net)
ls_forecast_co_ex = find_potential_nodes(ls_adv, ls_forbidden, weight_co_ex, df_nodes_co_ex_1)
df_forecast_chain = pd.DataFrame(ls_forecast_chain, columns=['t', 'origin', 'destination'])
df_forecast_co_ex = pd.DataFrame(ls_forecast_co_ex, columns=['t', 'origin', 'destination'])
df_result_chain = df_forecast_chain.loc[df_forecast_chain['t'] <= search_step_]
df_result_co_ex = df_forecast_co_ex.loc[df_forecast_co_ex['t'] <= search_step_]
ls_pre_chain = df_result_chain['destination'].unique()
ls_pre_co_ex = df_result_co_ex['destination'].unique()
ls_pre = [x for x in ls_pre_chain if x in ls_pre_co_ex]  # 共现求得的模拟结果和上下游求得的模拟结果取交集
ls_pre = [x for x in ls_pre if x not in ls_local_best and x not in ls_forbidden]
ls_pre = list(set(ls_pre))

# 根据是否全联通参数，确定是否需要进行全联通判断
if if_connection_ == '是':
    # 将模拟出的节点加入到之前设置的种子产品和优势产品中
    ls_chain_local_nodes_ = ls_adv + ls_final
    # 用得到的节点编码列表（模拟条件优势产品+种子产品+模拟结果产品）构成邻接矩阵，以此计算可达矩阵，判断这些点是否全连通
    adj_matrix_chain, member_dict_chain = adjacence_matrix(ls_chain_local_nodes_, df_net)
    adj_matrix_co_ex, member_dict_co_ex = adjacence_matrix(ls_chain_local_nodes_, df_nodes_co_ex_1)
    # print(adj_matrix)
    reach_matrix_chain = get_reach_matrix(adj_matrix_chain)  # 上下游到达矩阵
    reach_matrix_co_ex = get_reach_matrix(adj_matrix_co_ex)  # 共现到达矩阵

    # 当上下游或共现的可达矩阵中，有一个值为0的元素个数为0，则说明这些节点构成的网络已经达到全连通。
    if len(np.where(reach_matrix_chain == 0)[0]) == 0 or len(np.where(reach_matrix_co_ex == 0)[0]) == 0:
        print('模拟结果已构成全联通网络，直接输出结果。')

    else:
        ls_zero_chain = {}
        for i in range(len(reach_matrix_chain)):
            ls_i = reach_matrix_chain[i].tolist()[0]
            ls_zero_chain[i] = len([x for x in ls_i if x == 0])
        min_index_chain = []
        for key, item in ls_zero_chain.items():
            if item == min(ls_zero_chain.values()):
                min_index_chain.append(key)
            # print(key, item)

        ls_zero_co_ex = {}
        for i in range(len(reach_matrix_co_ex)):
            ls_i = reach_matrix_co_ex[i].tolist()[0]
            ls_zero_co_ex[i] = len([x for x in ls_i if x == 0])
        min_index_co_ex = []
        for key, item in ls_zero_co_ex.items():
            if item == min(ls_zero_co_ex.values()):
                min_index_co_ex.append(key)

        if min(ls_zero_co_ex.values()) < min(ls_zero_chain.values()):
            reach_matrix = reach_matrix_co_ex
            min_index_z = min_index_co_ex
            member_dict = member_dict_co_ex
        else:
            reach_matrix = reach_matrix_chain
            min_index_z = min_index_chain
            member_dict = member_dict_chain

        # 当预测出的产品与模拟条件产品并没有构成全连通时，需要通过算法寻找补充节点，find_shortest_path_算法慢的不行
        if path_calculation == '最短路径':
            # 首先找到没有形成全联通的点对有哪些
            location = np.where(reach_matrix == 0)
            path_total = []
            path_weight_total = []
            ls_add = []
            for i in range(len(location[0])):
                start_index = location[0][i]
                end_index = location[1][i]
                if end_index > start_index == min_index_z[0]:
                    start_node = list(member_dict.keys())[start_index]
                    end_node = list(member_dict.keys())[end_index]
                    # 用最短路径算法找到对应的补充路径
                    path, path_weight = find_shortest_path_(start_node, end_node, all_nodes, ls_forbidden, df_net)
                    path_total.extend(path)
                    path_weight_total.append(path_weight)

                    ls_nodes = [y for x in path_total for y in x]
                    ls_nodes = list(Counter(ls_nodes))
                    ls_add_i = [x for x in ls_nodes if x not in ls_adv and x not in ls_final]
                    ls_add.extend(ls_add_i)
                    ls_add = list(Counter(ls_add))
                    ls_nodes = ls_final + ls_adv + ls_add
                    adj_matrix, member_dict_i = adjacence_matrix(ls_nodes, df_net)
                    reach_matrix = get_reach_matrix(adj_matrix)
                    if len(np.where(reach_matrix == 0)[0]) == 0:
                        # 当已经构成全联通时跳出循环得到最终结果，新增的节点ls_add用于构成全联通
                        print('已通过补充节点获得全联通')
                        break
                    # else:
                    #     print(start_index, end_index)
                    #     print(start_node, end_node)
                    #     print(len(np.where(reach_matrix == 0)[0]))
                    #     print(ls_add)
            ls_add = list(Counter(ls_add))

        elif path_calculation == '最小代价':
            # 首先找到没有形成全联通的点对有哪些
            location = np.where(reach_matrix == 0)
            path_total = []
            path_weight_total = []
            ls_add = []
            for i in range(len(location[0])):
                start_index = location[0][i]
                end_index = location[1][i]
                if end_index > start_index == min_index_z[0]:
                    start_node = list(member_dict.keys())[start_index]
                    end_node = list(member_dict.keys())[end_index]
                    # 用最小代价算法寻找全联通补充节点
                    path, path_weight = find_minimal_path_(start_node, end_node, all_nodes, ls_forbidden, df_net)
                    path_total.extend(path)
                    path_weight_total.append(path_weight)

                    ls_nodes = [y for x in path_total for y in x]
                    ls_nodes = list(Counter(ls_nodes))
                    ls_add_i = [x for x in ls_nodes if x not in ls_adv and x not in ls_final]
                    ls_add.extend(ls_add_i)
                    ls_add = list(Counter(ls_add))
                    ls_nodes = ls_final + ls_adv + ls_add
                    adj_matrix, member_dict_i = adjacence_matrix(ls_nodes, df_net)
                    reach_matrix = get_reach_matrix(adj_matrix)
                    if len(np.where(reach_matrix == 0)[0]) == 0:
                        # 当已经构成全联通时跳出循环得到最终结果，新增的节点ls_add用于构成全联通
                        print('已通过补充节点获得全联通')
                        break
                    # else:
                    #     print(start_index, end_index)
                    #     print(start_node, end_node)
                    #     print(len(np.where(reach_matrix == 0)[0]))
                    #     print(ls_add)
            ls_add = list(Counter(ls_add))

        # 得到补充节点后，针对补充节点进行评估，与正式模拟出的节点评估分开
        ls_final_add = []
        for k in ls_add:
            ls_i = [k, code_to_name(k, df_nodes), code_to_chain(k, df_nodes)]
            t_k = 1
            # 然后找到该节点几步以内有多少本地节点
            ls_connect_k = find_connect_nodes(k, t_k, df_net)
            df_k_local = df_nodes.loc[(df_nodes['Id'].isin(ls_connect_k)) & (df_nodes['local'] == '1')]
            ls_i.append(len(df_k_local))

            # 看这些本地节点分别属于哪条产业链
            ls_chain_ = []
            ls_chain_k = df_k_local['chain'].tolist()
            for b in ls_chain_k:
                if len(b) > 1:
                    # print(b)
                    chain = json.loads(b.replace("'", '"'))
                    ls_chain_.extend(chain)
            ls_i.append(list(set(ls_chain_)))
            ls_final_add.append(ls_i)

        # 打印查看补充产品节点结果
        df_final_add = pd.DataFrame(ls_final_add, columns=['product_code',
                                                           'product_name', 'chain', 'local_support', 'loc_sup_chain'])
        df_final_add.sort_values(['local_support'], ascending=False, inplace=True)
        print('补充结果评价', len(df_final_add), df_final_add)

elif if_connection_ == '否' and path_calculation == '无':
    print('模拟结果不需要全联通，直接输出结果。')

# 当已经构成全连通网络，则不需要新增节点，与全联通参数选否的情况一样，直接对当前模拟出来的节点按照本地支持力度进行排序
# 根据预测到的节点编码，补充该节点属于哪条或哪几条产业链，以及其本地关联产品属于哪些产业链
ls_final_chain = []
for k in ls_final:
    ls_i = [k, code_to_name(k, df_nodes), code_to_chain(k, df_nodes)]
    t_k = 1
    # 然后找到该节点几步以内有多少本地节点
    ls_connect_k = find_connect_nodes(k, t_k, df_net)
    df_k_local = df_nodes.loc[(df_nodes['Id'].isin(ls_connect_k)) & (df_nodes['local'] == '1')]
    ls_i.append(len(df_k_local))

    # 看这些本地节点分别属于哪条产业链
    ls_chain_ = []
    ls_chain_k = df_k_local['chain'].tolist()
    for b in ls_chain_k:
        if len(b) > 1:
            # print(b)
            chain = json.loads(b.replace("'", '"'))
            ls_chain_.extend(chain)
    ls_i.append(list(set(ls_chain_)))
    ls_final_chain.append(ls_i)

# 打印查看模拟出的属于该产业链的产品节点结果
df_final = pd.DataFrame(ls_final_chain,
                        columns=['product_code', 'product_name', 'chain', 'local_support', 'loc_sup_chain'])
df_final.sort_values(['local_support'], ascending=False, inplace=True)

print('模拟结果评价', len(df_final), df_final)

# 查询模拟结果的模拟链路
    node_code = input('请输入要查询的产品编码（产品编码/全部）：')
    path_check = []
    path_check_name = []
    if node_code == '全部':
        for i in ls_final:
            path_i = full_path_(df_forecast_chain, i)
            path_check.extend(path_i)
            path_i_name = []
            for a in path_i:
                ls_b = []
                for b in a:
                    name_b = code_to_name(b, df_nodes)
                    ls_b.append(name_b)
                path_i_name.append(ls_b)
            path_check_name.extend(path_i_name)
    else:
        ls_end_nodes = json.loads(node_code)
        for i in ls_end_nodes:
            path_i = full_path_(df_forecast_chain, i)
            path_check.extend(path_i)
            path_i_name = []
            for a in path_i:
                ls_b = []
                for b in a:
                    name_b = code_to_name(b, df_nodes)
                    ls_b.append(name_b)
                path_i_name.append(ls_b)
            path_check_name.extend(path_i_name)
    path_result = []
    for i in range(len(path_check)):
        path_result_i = [path_check[i], path_check_name[i]]
        print(pd.DataFrame(path_result_i))
        path_result.append(path_result_i)

请输入模拟模式（快速模拟/高级模拟）：高级模拟
请输入模拟场景(补短板/锻长板/促转型/建集群/自定义)：自定义
请输入产业链名称：["羊子"]
是否选择预置的优势产品（是/否）：否
请输入要参与模拟的优势产品：["羊"]
请输入不参与模拟的优势产品：[]
请输入参与模拟的种子产品：[]
是否使用预置的排除产品（是/否）：是
请输入要增加的排除产品：[]
请输入要减少的排除产品：[]
请输入参与模拟的联系强度（最强/较强/中等/较弱/最弱）：较强
请输入模拟的搜索半径（一步/两步/三步/四步/五步）：两步
请输入模拟结果是否全联通（是/否）：是
请输入算法选择（最短路径/最小代价）：最短路径
补充结果评价 0 Empty DataFrame
Columns: [product_code, product_name, chain, local_support, loc_sup_chain]
Index: []
模拟结果评价 2   product_code product_name    chain  local_support       loc_sup_chain
1     14110501        食品防腐剂        0             22  [羊子产业链, 黄河沿岸枣业产业链]
0     14040204       干酪（奶酪）  [羊子产业链]              5             [羊子产业链]
是否对模拟结果有疑问（是/否）：是
请输入要查询的产品编码（产品编码/全部）：全部
       0        1         2
0  30106  1440010  14040204
1      羊      乳制品    干酪（奶酪）
       0        1         2
0  30106  1440010  14110501
1      羊      乳制品     食品防腐剂


In [29]:
ls_pre_co_ex

array(['13020202', '30102', '1302', ..., '7409100000', '8301040101',
       '65010301'], dtype=object)

In [25]:
ls_pre

['14040204', '14110501']

In [21]:
ls_pre_chain

array(['1722030', '1721030', '1702029901', '1702029902', '17020201',
       '13070203', '1440010', '303010200', '3030501', '3030502',
       '303050202', '301060300', '28030201', '362808', '362809', '362810',
       '171202', '1821710', '1811060302', '17050602', '1702030101',
       '1821010402', '17', '3628030101', '7503030000', '7611030000',
       '1811750', '1353010', '14110501', '1307', '1440070', '14040204'],
      dtype=object)

In [31]:
df_forecast_chain.loc[df_forecast_chain['t'] == 1]

,t,origin,destination
0,1,30106,1722030
1,1,30106,1721030
2,1,30106,1702029901
3,1,30106,1702029902
4,1,30106,17020201
5,1,30106,13070203
6,1,30106,1440010
7,1,30106,303010200
8,1,30106,3030501
9,1,30106,3030502


In [24]:

nodes_ls = []
for x in ls_pre_chain:
    df_x = df_nodes.loc[df_nodes['Id'] == x]
    code_x = df_x['Label'].tolist()[0]
    print(code_x)
    nodes_ls.append(code_x)

毛机织物（呢绒）
毛纱
精梳羊绒纱线
粗梳羊绒纱线
羊毛纱
冻羊肉
乳制品
生羊奶
绵羊毛
山羊毛
山羊绒
能繁殖母羊
涤纶短纤维
粗纱机
细纱机
纺织纱线处理机械
针织钩编织物（针织坯布）
针织服装
羊毛针织休闲衫
毛制毯
针织纯毛绒线
毛针织帽
纺织产品
洗毛机
畜牧、兽医研究服务
时装设计服务
西服套装
熟肉制品
食品防腐剂
畜禽肉、油脂及食用杂碎
乳粉
干酪（奶酪）


In [22]:
ls_pre_co_ex

array(['13020202', '30102', '1302', ..., '7409100000', '8301040101',
       '65010301'], dtype=object)

In [23]:
ls_pre

['14040204', '14110501']

In [17]:
ls_adv

['30106']

In [17]:
len(ls_final)

164

In [32]:
ls_chain_local_nodes_ = ls_adv + ls_final
adj_matrix_chain, member_dict_chain = adjacence_matrix(ls_chain_local_nodes_, df_net)
adj_matrix_co_ex, member_dict_co_ex = adjacence_matrix(ls_chain_local_nodes_, df_nodes_co_ex_1)
# print(adj_matrix)
reach_matrix_chain = get_reach_matrix(adj_matrix_chain)  # 上下游到达矩阵
reach_matrix_co_ex = get_reach_matrix(adj_matrix_co_ex)  # 共现到达矩阵

In [31]:
reach_matrix = reach_matrix_chain
min_index_z = min_index_chain
member_dict = member_dict_chain
df_net_temp = df_net

In [41]:
if min(ls_zero_co_ex.values()) < min(ls_zero_chain.values()):
    reach_matrix = reach_matrix_co_ex
    min_index_z = min_index_co_ex
    member_dict = member_dict_co_ex
    df_net_temp = df_nodes_co_ex_1
else:
    reach_matrix = reach_matrix_chain
    min_index_z = min_index_chain
    member_dict = member_dict_chain
    df_net_temp = df_net

In [45]:
location = np.where(reach_matrix == 0)
path_total = []
path_weight_total = []
ls_add = []

In [76]:
for i in range(len(location[0])):
    start_index = location[0][i]
    end_index = location[1][i]
    if end_index > start_index == min_index_z[0]:
        start_node = list(member_dict.keys())[start_index]
        end_node = list(member_dict.keys())[end_index]
        # 用最短路径算法找到对应的补充路径
        print(i)
        print(start_node,end_node)
        path, path_weight = find_shortest_path_(start_node, end_node, all_nodes, ls_forbidden, df_net_temp)
        path_total.extend(path)
        path_weight_total.append(path_weight)

        ls_nodes = [y for x in path_total for y in x]
        ls_nodes = list(Counter(ls_nodes))
        ls_add_i = [x for x in ls_nodes if x not in ls_adv and x not in ls_final]
        ls_add.extend(ls_add_i)
        ls_add = list(Counter(ls_add))
        ls_nodes = ls_final + ls_adv + ls_add
        adj_matrix_i, member_dict_i = adjacence_matrix(ls_nodes, df_net_temp)
        reach_matrix = get_reach_matrix(adj_matrix_i)
        if len(np.where(reach_matrix == 0)[0]) == 0:
            # 当已经构成全联通时跳出循环得到最终结果，新增的节点ls_add用于构成全联通
            print('已通过补充节点获得全联通')
            break
        # else:
        #     print(start_index, end_index)
        #     print(start_node, end_node)
        #     print(len(np.where(reach_matrix == 0)[0]))
        #     print(ls_add)
ls_add = list(Counter(ls_add))

0
1011010 620201
1
1011010 2601060200
2
1011010 170103
3
1011010 3206130201
4
1011010 2603060101
5
1011010 631302
6
1011010 1701050301
7
1011010 9202010000
8
1011010 650203
9
1011010 7801010100
10
1011010 6201
11
1011010 1704020101
12
1011010 620103
13
1011010 1701050302
14
1011010 58
15
1011010 7410
16
1011010 1701050304
17
1011010 6201020000
18
1011010 1701050303
19
1011010 8201010202
20
1011010 1704020199
21
1011010 1704020104
22
1011010 1704020102
23
1011010 1704020103
24
1011010 620202
25
1011010 6320050900
已通过补充节点获得全联通


In [15]:
question = input('是否对模拟结果有疑问（是/否）：')
if question == '是':
    node_code = input('请输入要查询的产品编码（产品编码/全部）：')  # 产品编码可用["2342","3245"]的格式进行多选
    path_check = []
    path_check_name = []
    if node_code == '全部':
        for i in ls_final:
            path_i = full_path_(df_forecast_chain, i)
            path_check.extend(path_i)
            path_i_name = []
            for a in path_i:
                ls_b = []
                for b in a:
                    name_b = code_to_name(b, df_nodes)
                    ls_b.append(name_b)
                path_i_name.append(ls_b)
            path_check_name.extend(path_i_name)
    path_result = []
    for i in range(len(path_check)):
        path_result_i = [path_check[i], path_check_name[i]]
        print(pd.DataFrame(path_result_i))
        path_result.append(path_result_i)

是否对模拟结果有疑问（是/否）：是
请输入要查询的产品编码（产品编码/全部）：全部
         0         1         2
0  2651010  63190213    300109
1   初级形态塑料  橡胶制品批发服务  塑料包装箱及容器
         0         1         2
0  2651010  63190214    300109
1   初级形态塑料  塑料制品批发服务  塑料包装箱及容器
         0         1
0  2651010  26130108
1   初级形态塑料      环氧树脂
         0           1         2
0  3462010  3610030100  32080234
1       风机       木材干燥机   不锈钢大型型钢
         0           1         2
0  3462010  3618010211  32080234
1       风机         干燥机   不锈钢大型型钢
         0           1         2
0  3462010  3604090200  32080234
1       风机       转筒干燥机   不锈钢大型型钢
         0           1         2
0  3462010  3604090400  32080234
1       风机       矿石干燥机   不锈钢大型型钢
         0         1         2
0  3462010  35300401  32080234
1       风机      干燥设备   不锈钢大型型钢
         0           1         2
0  3462010  3608120100  32080234
1       风机       辊式干燥机   不锈钢大型型钢
         0         1
0  2651010  63190213
1   初级形态塑料  橡胶制品批发服务
         0            1           2
0  3130710       36310

1       钢材  制浆和造纸专用设备         卫生纸
         0        1
0  3130710   350701
1       钢材  锻造机及冲压机
         0            1           2
0  3130710       363902  1304010201
1       钢材  饲料（草）加工机械设备       猪浓缩饲料
         0         1           2
0  3130710   3534010  1304010201
1       钢材  饲料生产专用设备       猪浓缩饲料
      0           1           2
0    74  7702010000  6201020000
1  商务服务      技术开发服务     数据库软件产品
         0        1     2
0  2651010     3917    67
1   初级形态塑料  家用厨房电器具  餐饮服务
         0        1           2
0  2820010  1800010  1701050303
1     合成纤维       服装  化学纤维短纤混纺平布
         0       1           2
0  2651010  260202  2502120204
1   初级形态塑料      环烃        微晶石蜡
         0        1           2
0  2651010   260213  2502120204
1   初级形态塑料  羧酸及其衍生物        微晶石蜡
         0         1           2
0  2651010    260209  2502120204
1   初级形态塑料  无环醇及其衍生物        微晶石蜡
         0       1           2
0  2651010    2602  2502120204
1   初级形态塑料  有机化学原料        微晶石蜡
         0       1             2
0  2651010  260

In [53]:
end_index

4

In [93]:
start_node

'3011030'

In [94]:
end_node

'2619040205'

In [59]:
s_1 = [[start_node]]
s_2 = [start_node]
# 初始化剩余点集合
u = [x for x in all_nodes if x not in s_2]
t = 1
ls_forecast = []

In [60]:
while end_node in u:
    ls_nodes_temp_ = []
    df_temp_ = df_nodes_co_ex_1.loc[(df_nodes_co_ex_1['source'].isin(s_1[-1])) | (df_nodes_co_ex_1['target'].isin(s_1[-1]))]
    if len(df_temp_) > 0:
        ls_temp_ = df_temp_.loc[:, ['source', 'target']].values.tolist()
        ls_temp_ = [a for x in ls_temp_ for a in x]
        ls_temp_ = list(Counter(ls_temp_))
        ls_temp_ = [x for x in ls_temp_ if x not in s_2 and x not in ls_forbidden]
        ls_temp_ = list(Counter(ls_temp_))

        for j in ls_temp_:
            ls_forecast_i = []
            df_weight = df_temp_.loc[(df_temp_['source'] == j) | (df_temp_['target'] == j)]
            weight = np.mean(df_weight['weight'].tolist())
            ls_forecast_i.extend([t, i, j, weight])
            ls_forecast.append(ls_forecast_i)
        ls_nodes_temp_.extend(ls_temp_)
    else:
        continue
    if len(ls_nodes_temp_) == 0:
        print('no connection')
        break
    else:
        s_1.append(ls_nodes_temp_)
        s_2.extend(ls_nodes_temp_)
        t += 1
        u = [x for x in all_nodes if x not in s_2]
        print('xxxxxxxxxx',t, len(ls_nodes_temp_),'xxxxxxxxxx')

xxxxxxxxxx 2 15 xxxxxxxxxx
xxxxxxxxxx 3 322 xxxxxxxxxx
xxxxxxxxxx 4 2787 xxxxxxxxxx


In [72]:
start_node = '1011010'
end_node = '620201'
s_1 = [[start_node]]
s_2 = [start_node]
# 初始化剩余点集合
u = [x for x in all_nodes if x not in s_2]
t = 1
ls_forecast = []

In [65]:
while end_node in u:
    ls_nodes_temp_ = []
    for i in s_1[-1]:
        df_temp_ = df_net.loc[(df_net['source'] == i) | (df_net['target'] == i)]
        if len(df_temp_) > 0:
            ls_temp_ = df_temp_.loc[:, ['source', 'target']].values.tolist()
            ls_temp_ = [a for x in ls_temp_ for a in x]
            ls_temp_ = [x for x in ls_temp_ if x not in s_2 and x not in ls_forbidden]
            ls_temp_ = list(Counter(ls_temp_))

            for j in ls_temp_:
                ls_forecast_i = []
                df_weight = df_temp_.loc[(df_temp_['source'] == j) | (df_temp_['target'] == j)]
                weight = np.mean(df_weight['weight'].tolist())
                ls_forecast_i.extend([t, i, j, weight])
                ls_forecast.append(ls_forecast_i)
            ls_nodes_temp_.extend(ls_temp_)
        else:
            continue
    ls_nodes_temp_ = list(Counter(ls_nodes_temp_))
    if len(ls_nodes_temp_) == 0:
        break
    else:
        s_1.append(ls_nodes_temp_)
        s_2.extend(ls_nodes_temp_)
        t += 1
        u = [x for x in all_nodes if x not in s_2]
        print('xxxxxxxxxx',t, len(ls_nodes_temp_),'xxxxxxxxxx')

xxxxxxxxxx 2 11 xxxxxxxxxx
xxxxxxxxxx 3 99 xxxxxxxxxx
xxxxxxxxxx 4 728 xxxxxxxxxx
xxxxxxxxxx 5 1867 xxxxxxxxxx
xxxxxxxxxx 6 688 xxxxxxxxxx


In [67]:
df_forecast = pd.DataFrame(ls_forecast, columns=['t', 'origin', 'destination', 'weight'])
location_end = np.where(df_forecast == end_node)

,t,origin,destination,weight
20496,5,3911010,620201,3.5
20526,5,3911020,620201,3.5
20556,5,4009010101,620201,3.5
20586,5,4009010102,620201,3.5
20616,5,3911030,620201,3.5
20646,5,3911040,620201,3.5
20676,5,3911050,620201,3.5
20706,5,4009010203,620201,3.5
20736,5,4009010204,620201,3.5
20766,5,4009010205,620201,3.5


In [68]:
max_t = max(df_forecast['t'].tolist())
t_temp = max_t
path = [[end_node]]
while t_temp >= 1:
    path_a = []
    for i in path:
        df_temp = df_forecast.loc[(df_forecast['t'] == t_temp) & (df_forecast['destination'] == i[0])]
        df_temp_ls = df_temp['origin'].tolist()

        for a in df_temp_ls:
            ls_a = [a]
            ls_a.extend(i)
            path_a.append(ls_a)

    path = path_a
    t_temp -= 1

# 根据path列表求得每条路径对应的代价之和
path_weight = []
for i in path:
    weight_i = []
    for j in range(len(i) - 1):
        df_weight = df_forecast.loc[(df_forecast['origin'] == i[j]) & (df_forecast['destination'] == i[j + 1])]
        weight_ij = df_weight['weight'].tolist()[0]
        weight_i.append(1 / weight_ij)
    path_weight.append(sum(weight_i))
# 根据path_weight列表找到代价最小的那条路径
path_weight_min = min(path_weight)
path_min = [path[x] for x in range(len(path)) if path_weight[x] == path_weight_min]

In [70]:
path_weight_min

1.669047619047619

In [69]:
path_min

[['1011010', '3011010', '3216010', '3973710', '3911010', '620201'],
 ['1011010', '3011010', '3216010', '3973710', '3911020', '620201'],
 ['1011010', '3011010', '3216010', '3973710', '4009010101', '620201'],
 ['1011010', '3011010', '3216010', '3973710', '4009010102', '620201'],
 ['1011010', '3011010', '3216010', '3973710', '3911030', '620201'],
 ['1011010', '3011010', '3216010', '3973710', '3911040', '620201'],
 ['1011010', '3011010', '3216010', '3973710', '3911050', '620201'],
 ['1011010', '3011010', '3216010', '3973710', '4009010203', '620201'],
 ['1011010', '3011010', '3216010', '3973710', '4009010204', '620201'],
 ['1011010', '3011010', '3216010', '3973710', '4009010205', '620201'],
 ['1011010', '3011010', '3216010', '3973710', '3911060', '620201'],
 ['1011010', '3011010', '3216010', '3973710', '4009010299', '620201']]

In [75]:
start_node = '1011010'
end_node = '620201'
find_shortest_path_(start_node, end_node, all_nodes, ls_forbidden, df_net)

([['1011010', '3011010', '3216010', '3973710', '3911010', '620201'],
  ['1011010', '3011010', '3216010', '3973710', '3911020', '620201'],
  ['1011010', '3011010', '3216010', '3973710', '4009010101', '620201'],
  ['1011010', '3011010', '3216010', '3973710', '4009010102', '620201'],
  ['1011010', '3011010', '3216010', '3973710', '3911030', '620201'],
  ['1011010', '3011010', '3216010', '3973710', '3911040', '620201'],
  ['1011010', '3011010', '3216010', '3973710', '3911050', '620201'],
  ['1011010', '3011010', '3216010', '3973710', '4009010203', '620201'],
  ['1011010', '3011010', '3216010', '3973710', '4009010204', '620201'],
  ['1011010', '3011010', '3216010', '3973710', '4009010205', '620201'],
  ['1011010', '3011010', '3216010', '3973710', '3911060', '620201'],
  ['1011010', '3011010', '3216010', '3973710', '4009010299', '620201']],
 1.669047619047619)

In [74]:
def find_shortest_path_(start_node, end_node, all_nodes, ls_forbidden, df_net):
    # start_node是起点产品节点的编码，str
    # end_node是终点产品节点的编码，str
    # all_nodes是图谱里的所有产品节点，包含起点和终点，list
    # ls_forbidden是设定好的排除产品，不会出现在模拟结果中
    # df_net是全国产品节点间联系数据，dataframe

    # 初始化起点集合
    s_1 = [[start_node]]
    s_2 = [start_node]
    # 初始化剩余点集合
    u = [x for x in all_nodes if x not in s_2]
    t = 1
    ls_forecast = []

    while end_node in u:
        ls_nodes_temp_ = []
        for i in s_1[-1]:
            df_temp_ = df_net.loc[(df_net['source'] == i) | (df_net['target'] == i)]
            if len(df_temp_) > 0:
                ls_temp_ = df_temp_.loc[:, ['source', 'target']].values.tolist()
                ls_temp_ = [a for x in ls_temp_ for a in x]
                ls_temp_ = [x for x in ls_temp_ if x not in s_2 and x not in ls_forbidden]
                ls_temp_ = list(Counter(ls_temp_))

                for j in ls_temp_:
                    ls_forecast_i = []
                    df_weight = df_temp_.loc[(df_temp_['source'] == j) | (df_temp_['target'] == j)]
                    weight = np.mean(df_weight['weight'].tolist())
                    ls_forecast_i.extend([t, i, j, weight])
                    ls_forecast.append(ls_forecast_i)
                ls_nodes_temp_.extend(ls_temp_)
            else:
                continue
        ls_nodes_temp_ = list(Counter(ls_nodes_temp_))
        if len(ls_nodes_temp_) == 0:
            return [], []
        else:
            s_1.append(ls_nodes_temp_)
            s_2.extend(ls_nodes_temp_)
            t += 1
            u = [x for x in all_nodes if x not in s_2]
            # print(t, ls_nodes_temp_)
    # df_forecast 是最少步数可以从起点走到终点的数据集合
    df_forecast = pd.DataFrame(ls_forecast, columns=['t', 'origin', 'destination', 'weight'])
    location_end = np.where(df_forecast == end_node)
    if len(location_end[0]) != 0:
        # 根据路径数据，还原从起点到终点的路径path
        max_t = max(df_forecast['t'].tolist())
        t_temp = max_t
        path = [[end_node]]
        while t_temp >= 1:
            path_a = []
            for i in path:
                df_temp = df_forecast.loc[(df_forecast['t'] == t_temp) & (df_forecast['destination'] == i[0])]
                df_temp_ls = df_temp['origin'].tolist()

                for a in df_temp_ls:
                    ls_a = [a]
                    ls_a.extend(i)
                    path_a.append(ls_a)

            path = path_a
            t_temp -= 1

        # 根据path列表求得每条路径对应的代价之和
        path_weight = []
        for i in path:
            weight_i = []
            for j in range(len(i) - 1):
                df_weight = df_forecast.loc[(df_forecast['origin'] == i[j]) & (df_forecast['destination'] == i[j + 1])]
                weight_ij = df_weight['weight'].tolist()[0]
                weight_i.append(1 / weight_ij)
            path_weight.append(sum(weight_i))
        # 根据path_weight列表找到代价最小的那条路径
        path_weight_min = min(path_weight)
        path_min = [path[x] for x in range(len(path)) if path_weight[x] == path_weight_min]
        return path_min, path_weight_min
    else:
        return [], []

In [101]:
# 最短路径法找到路径
def find_shortest_path_(start_node,end_node,all_nodes,ls_forbidden,df_net):
    
    # 初始化起点集合
    s_1 = [[start_node]]
    s_2 = [start_node]
    # 初始化剩余点集合
    u = [x for x in all_nodes if x not in s_2]
    t = 1
    ls_forecast = []

    while end_node in u:
        ls_nodes_temp_ = []
        for i in s_1[-1]:
            df_temp_ = df_net.loc[(df_net['source'] == i)|(df_net['target'] == i)]
            if len(df_temp_) > 0:
                ls_temp_ = df_temp_.loc[:,['source','target']].values.tolist()
                ls_temp_ = [a for x in ls_temp_ for a in x]
                ls_temp_ = list(Counter(ls_temp_))
                ls_temp_ = [x for x in ls_temp_ if x not in s_2 and x not in ls_forbidden]
                ls_temp_ = list(Counter(ls_temp_))

                for j in ls_temp_:
                    ls_forecast_i = []
                    df_weight = df_temp_.loc[(df_temp_['source'] == j) | (df_temp_['target'] == j)]
                    weight = np.mean(df_weight['weight'].tolist())
                    ls_forecast_i.extend([t,i,j,weight])
                    ls_forecast.append(ls_forecast_i)
                ls_nodes_temp_.extend(ls_temp_)
            else:
                continue
        if len(ls_nodes_temp_) == 0:
            return [],[]
        ls_nodes_temp_ = list(Counter(ls_nodes_temp_))
        s_1.append(ls_nodes_temp_)
        s_2.extend(ls_nodes_temp_)
        t += 1
        u = [x for x in all_nodes if x not in s_2]
    # df_forecast 是最少步数可以从起点走到终点的数据集合
    df_forecast = pd.DataFrame(ls_forecast,columns=['t','origin','destination','weight'])
    location_end = np.where(df_forecast == end_node)
    if len(location_end[0]) != 0:
        # 根据路径数据，还原从起点到终点的路径path
        max_t = max(df_forecast['t'].tolist())
        t_temp = max_t
        path = []
        path.append([end_node])
        while t_temp >= 1:
            path_a = []
            for i in path:
                df_temp = df_forecast.loc[(df_forecast['t'] == t_temp) & (df_forecast['destination'] == i[0])]
                df_temp_ls = df_temp['origin'].tolist()

                for a in df_temp_ls:
                    ls_a = []
                    ls_a.append(a)
                    ls_a.extend(i)
                    path_a.append(ls_a)

            path = path_a
            t_temp -= 1

        # 根据path列表求得每条路径对应的代价之和
        path_weight = []
        for i in path:
            weight_i = []
            for j in range(len(i)-1):
                df_weight = df_forecast.loc[(df_forecast['origin'] == i[j]) & (df_forecast['destination'] == i[j+1])]
                weight_ij = df_weight['weight'].tolist()[0]
                weight_i.append(1/weight_ij)
            path_weight.append(sum(weight_i))
        # 根据path_weight列表找到代价最小的那条路径
        path_weight_min = min(path_weight)
        path_min = [path[x] for x in range(len(path)) if path_weight[x] == path_weight_min]
        return path_min,path_weight_min
    else:
        return [],[]

In [102]:
a = '1741030' # 绢纺丝
b = '4017010301' # 6英寸集成圆片，线宽0.8微米及以下
path, path_weight = find_shortest_path_(a, b, all_nodes,ls_forbidden,df_nodes_co_ex_1)

In [106]:
a = '3011030' 
b = '2619040205' 
path, path_weight = find_shortest_path_(a, b, all_nodes,ls_forbidden,df_nodes_co_ex_1)

In [107]:
path

[['3011030', '3031070', '1019010', '36', '2619040205'],
 ['3011030', '3031010', '1019010', '36', '2619040205'],
 ['3011030', '3031070', '1004030102', '36', '2619040205'],
 ['3011030', '3031010', '1004030102', '36', '2619040205'],
 ['3011030', '3031070', '10040102', '36', '2619040205'],
 ['3011030', '3031010', '10040102', '36', '2619040205'],
 ['3011030', '3031070', '100403', '36', '2619040205'],
 ['3011030', '3031010', '100403', '36', '2619040205']]

In [11]:
code_to_name('2664020', df_nodes)

'多晶硅'

In [13]:
ls_final

['3330030201',
 '2602130302',
 '3431',
 '4017010202',
 '6320050900',
 '470101',
 '331102',
 '39080103',
 '4017010301',
 '3209020111',
 '2613010407',
 '140603',
 '4017010201',
 '2613010802',
 '4017030403',
 '2602131103',
 '2602010205',
 '390802',
 '3111',
 '90101',
 '2602040100',
 '340101',
 '3632050301',
 '26180903',
 '63190213',
 '3205112002',
 '320613',
 '3206130201',
 '26021304',
 '2602130402',
 '320901',
 '1004030102',
 '3529010500',
 '2602020210',
 '2602160100',
 '2603060101',
 '3411',
 '26010101',
 '3111010100',
 '3208171200',
 '4017010102',
 '261305',
 '39070303',
 '331101',
 '32080233',
 '26010109',
 '2602020101',
 '343101',
 '32090103',
 '26130107',
 '4301040100',
 '2503040200',
 '26021303',
 '3208114100',
 '63190214',
 '2602110110',
 '3410',
 '3908030100',
 '360701',
 '4103100300',
 '26',
 '261404',
 '3001011703',
 '3908020100',
 '2601100309',
 '4017010302',
 '3529010400',
 '3634020100',
 '4001',
 '35380102',
 '3908020200',
 '2602130406',
 '30010106',
 '350701',
 '4017010203'

In [14]:
full_path_(df_forecast_chain, '4301040100')

[['3211020', '3240020', '4301040100']]

In [19]:
path_check = []
path_check_name = []
for i in ls_final:
    path_i = full_path_(df_forecast_chain, i)
    path_check.extend(path_i)
    path_i_name = []
    for a in path_i:
        ls_b = []
        for b in a:
            name_b = code_to_name(b, df_nodes)
            ls_b.append(name_b)
        path_i_name.append(ls_b)
    path_check_name.extend(path_i_name)
path_result = []
for i in range(len(path_check)):
    path_result_i = [path_check[i], path_check_name[i]]
    print(pd.DataFrame(path_result_i))
    path_result.append(path_result_i)

          0           1           2
0   3211020  3634020100  3330030201
1  精炼铜（电解铜）         电解槽         电解钴
          0        1           2
0   2614250  1462030  2602130302
1  冰乙酸（冰醋酸）       食醋          乙酸
          0        1           2
0   2614250  2614210  2602130302
1  冰乙酸（冰醋酸）      精甲醇          乙酸
          0         1           2
0   2614250  26021304  2602130302
1  冰乙酸（冰醋酸）       乙酸酯          乙酸
          0       1           2
0   2614250  140603  2602130302
1  冰乙酸（冰醋酸）  醋及醋代用品          乙酸
          0           1           2
0   2614250  2602130402  2602130302
1  冰乙酸（冰醋酸）       乙酸乙烯酯          乙酸
          0           1           2
0   2614250  2503040200  2602130302
1  冰乙酸（冰醋酸）        甲醇汽油          乙酸
         0        1
0  3130710     3431
1       钢材  金属链条及零件
         0                      1
0  2664020             4017010202
1      多晶硅  8英寸集成圆片，线宽0.18～0.35微米
         0       1           2
0  3130710    3524  6320050900
1       钢材  轴承及其零件      轴承批发服务
         0        1      

1      多晶硅  12英寸集成圆片，线宽90纳米及以下  通信传输设备
         0        1       2
0  2664020   390802    4001
1      多晶硅  电力微电子组件  通信传输设备
        0       1         2
0  250401  320901  35380102
1      焦炭   普通铁合金     球墨铸铁件
        0        1         2
0  250401  3140710  35380102
1      焦炭      铁合金     球墨铸铁件
         0           1
0  2664020  3908020200
1      多晶硅     电力半导体组件
          0        1           2
0   2614250  2614210  2602130406
1  冰乙酸（冰醋酸）      精甲醇        乙酸甲酯
          0           1           2
0   2614250  2503040200  2602130406
1  冰乙酸（冰醋酸）        甲醇汽油        乙酸甲酯
            0         1
0     3042010  30010106
1  太阳能工业用超白玻璃    聚酯塑料薄膜
         0        1
0  3130710   350701
1       钢材  锻造机及冲压机
         0           1
0  2664020  4017010203
1      多晶硅  线宽0.35微米以上
         0           1
0  2664020  2618090400
1      多晶硅        多晶硅片
          0        1          2
0   3211020  3240020  901010199
1  精炼铜（电解铜）      铜合金     其他矿含铜量
          0           1          2
0   3211020  2601100309  9010

In [10]:
path_result

[[['3211020', '3240020', '4301040100'], ['3211020', '3240020', '4301040100']],
 [['2664020', '363201', '26180903'], ['2664020', '363201', '26180903']],
 [['2614250', '140603'], ['2614250', '140603']],
 [['3130710', '360701'], ['3130710', '360701']],
 [['3211020', '3634020100', '3330030201'], ['精炼铜（电解铜）', '电解槽', '电解钴']],
 [['2614250', '1462030', '2602130302'], ['冰乙酸（冰醋酸）', '食醋', '乙酸']],
 [['2614250', '2614210', '2602130302'], ['冰乙酸（冰醋酸）', '精甲醇', '乙酸']],
 [['2614250', '26021304', '2602130302'], ['冰乙酸（冰醋酸）', '乙酸酯', '乙酸']],
 [['2614250', '140603', '2602130302'], ['冰乙酸（冰醋酸）', '醋及醋代用品', '乙酸']],
 [['2614250', '2602130402', '2602130302'], ['冰乙酸（冰醋酸）', '乙酸乙烯酯', '乙酸']],
 [['2614250', '2503040200', '2602130302'], ['冰乙酸（冰醋酸）', '甲醇汽油', '乙酸']],
 [['3130710', '3431'], ['钢材', '金属链条及零件']],
 [['2664020', '4017010202'], ['多晶硅', '8英寸集成圆片，线宽0.18～0.35微米']],
 [['3130710', '3524', '6320050900'], ['钢材', '轴承及其零件', '轴承批发服务']],
 [['3130710', '340101', '470101'], ['钢材', '钢结构及其产品', '房屋工程服务']],
 [['3211020', '331102

In [47]:
# 主程序
# 输入模拟模式
type_ = input('请输入模拟模式（快速模拟/高级模拟）：')

# 设定联系强度对应值
ls_connection_weight = ['最强', '较强', '中等', '较弱', '最弱']
ls_connection_weight_chain = [4.5, 3.5, 2.5, 1.5, 0.5]
ls_connection_weight_co_ex = [0.8, 0.6, 0.4, 0.2, 0.1]

if type_ == '快速模拟':
    env_ = '无'  # 模拟场景在快速模拟中为空
else:
    # 输入模拟场景：补短板、锻长板、促转型、建集群、自定义
    env_ = input('请输入模拟场景(补短板/锻长板/促转型/建集群/自定义)：')  # 模拟场景在高级模拟中可自行选择

# 输入模拟对象产业链，注：这里输入的产业链名称字符串必须用双引号，exp：["轻纺产业链"]，可以输入多条产业链，用英文格式的逗号连接
# 注：输入产业链时，可以只输入产业链名称的简称，比如轻纺、光伏、大宗固废等。
# 注：（当用户是非链长和非管理员用户时，不会输入预测对象产业链，而是直接输入优势产品及种子产品）
chain = input('请输入产业链名称：')
ls_chain = json.loads(chain)

# 找到模拟对象产业链（可能不止一条产业链）中所有本地节点
df_local_chain = pd.DataFrame([])
for n in ls_chain:
    df_chain_local_n = df_nodes.loc[(df_nodes['chain'].str.contains(n)) & (df_nodes['local'] == '1')]
    df_local_chain = pd.concat([df_local_chain, df_chain_local_n], axis=0)

df_local_chain = df_local_chain.drop_duplicates(inplace=False)
ls_chain_local_nodes = df_local_chain['Id'].tolist()

# 根据模拟产业链，得到这些产业链由哪些节点组成
ls_chain_nodes = find_chain_ls(ls_chain, df_nodes)

if env_ == '无':
    # 快速模拟，条件配置：
    ls_adv = ls_chain_local_nodes  # 优势产业为输入的产业链所有本地产品，种子产业为无
    # 排除产品名称：原铝（电解铝），铝压延加工材,铝材,铝合金,粗纱机,毛纱,羊毛纱
    # 排除产品代码：'331603','3336','333601','33340105','362808','170202','17020201'
    ls_forbidden = ['331603', '3336', '3316', '333601', '33340105', '362808', '170202', '17020201']
    index_weight = ls_connection_weight.index('较强')
    weight_chain = ls_connection_weight_chain[index_weight]  # 上下游阈值大于等于3.5(较强)
    weight_co_ex = ls_connection_weight_co_ex[index_weight]  # 上下游阈值大于等于0.6(较强)

    search_step_ = 2  # 步数小于等于2步
    if_connection_ = '否'  # 是否全连通：********否*********
    path_calculation = '无'  # 算法名称：无
    # 备注：税务数据涉及的产品节点优先，产业链内部产品节点优先
elif env_ != '无':
    # 高级模拟，条件配置：
    # 输入优势产品，是否选择预置优势产品,预置种子产品为产业链中的本地既有产品
    if_preset_adv = input('是否选择预置的优势产品（是/否）：')
    if if_preset_adv == '是':
        ls_adv = ls_chain_local_nodes
    else:
        ls_adv = []
    # 用户完善优势产品列表，输入为列表形式，exp：["电力电子元器件","水泥混凝土制品"]，可以输入多个产品，用英文格式的逗号连接
    adv_nodes_plus = input('请输入要参与模拟的优势产品：')
    adv_nodes_plus_ls = json.loads(adv_nodes_plus)
    adv_nodes_minus = input('请输入不参与模拟的优势产品：')
    adv_nodes_minus_ls = json.loads(adv_nodes_minus)
    ls_adv.extend(adv_nodes_plus_ls)
    ls_adv = list(set(ls_adv))
    ls_adv = [x for x in ls_adv if x not in adv_nodes_minus_ls]

    # 输入种子产品列表（选填内容），输入为列表形式，exp：["电力电子元器件","水泥混凝土制品"]，可以输入多个产品，用英文格式的逗号连接
    seed_nodes = input('请输入参与模拟的种子产品：')
    ls_seed = json.loads(seed_nodes)

    # 输入排除产品，是否选择预置的排除产品
    # 预置的排除产品名称：原铝（电解铝），铝，铝压延加工材,铝材,铝合金,粗纱机,毛纱,羊毛纱
    # 排除产品代码：'331603','3336','3316','333601','33340105','362808','170202','17020201'
    if_preset_forbid = input('是否使用预置的排除产品（是/否）：')
    if if_preset_forbid == '是':
        ls_forbidden = ['331603', '3336', '3316', '333601', '33340105', '362808', '170202', '17020201']
    else:
        ls_forbidden = []
    # 调整完善排除产品列表
    forbid_nodes_plus = input('请输入要增加的排除产品：')
    forbid_nodes_plus_ls = json.loads(forbid_nodes_plus)
    forbid_nodes_minus = input('请输入要减少的排除产品：')
    forbid_nodes_minus_ls = json.loads(forbid_nodes_minus)
    ls_forbidden.extend(forbid_nodes_plus_ls)
    ls_forbidden = list(set(ls_forbidden))
    ls_forbidden = [x for x in ls_forbidden if x not in forbid_nodes_minus_ls]  # 排除产品

    ls_adv = ls_adv + ls_seed
    ls_adv = list(set(ls_adv))  # 优势产品节点

    if env_ == '补短板' or env_ == '促转型' or env_ == '自定义':
        # 输入模拟阈值:最强4、较强3、中等2、较弱1、最弱0
        weight = input('请输入参与模拟的联系强度（最强/较强/中等/较弱/最弱）：')
        index_weight = ls_connection_weight.index(weight)
        weight_chain = ls_connection_weight_chain[index_weight]
        weight_co_ex = ls_connection_weight_co_ex[index_weight]

        # 输入搜索半径：一到五步
        step_ = input('请输入模拟的搜索半径（一步/两步/三步/四步/五步）：')
        step_1 = ['一步', '两步', '三步', '四步', '五步']
        step_2 = [1, 2, 3, 4, 5]
        step_ind = step_1.index(step_)
        search_step_ = step_2[step_ind]
    elif env_ == '锻长板':
        # 是否自定阈值
        if_define_weight = input('是否使用默认搜索阈值（是/否）:')
        if if_define_weight == '否':
            # 当用户选择自定义阈值时，需要输入对应的阈值强度，参与模拟
            weight = input('请输入参与模拟的联系强度（最强/较强/中等/较弱/最弱）：')
            index_weight = ls_connection_weight.index(weight)
            weight_chain = ls_connection_weight_chain[index_weight]
            weight_co_ex = ls_connection_weight_co_ex[index_weight]
        elif if_define_weight == '是':
            # 当用户选择内置的阈值时，按照
            index_weight = ls_connection_weight.index('较强')
            weight_chain = ls_connection_weight_chain[index_weight]
            weight_co_ex = ls_connection_weight_co_ex[index_weight]
        # 输入搜索半径：一到五步
        step_ = input('请输入模拟的搜索半径（一步/两步/三步/四步/五步）：')
        step_1 = ['一步', '两步', '三步', '四步', '五步']
        step_2 = [1, 2, 3, 4, 5]
        step_ind = step_1.index(step_)
        search_step_ = step_2[step_ind]
    elif env_ == '建集群':
        # 是否自定阈值
        if_define_weight = input('是否使用默认搜索阈值（是/否）:')
        if if_define_weight == '否':
            # 当用户选择自定义阈值时，需要输入对应的阈值强度，参与模拟
            weight = input('请输入参与模拟的联系强度（最强/较强/中等/较弱/最弱）：')
            index_weight = ls_connection_weight.index(weight)
            weight_chain = ls_connection_weight_chain[index_weight]
            weight_co_ex = ls_connection_weight_co_ex[index_weight]
        elif if_define_weight == '是':
            # 当用户选择内置的阈值时，按照
            index_weight = ls_connection_weight.index('最强')
            weight_chain = ls_connection_weight_chain[index_weight]
            weight_co_ex = ls_connection_weight_co_ex[index_weight]

        # 是否自定阈值搜索半径
        if_define_step = input('是否使用默认搜索半径（是/否）:')
        if if_define_step == '否':

            step_ = input('请输入模拟的搜索半径（一步/两步/三步/四步/五步）：')
            step_1 = ['一步', '两步', '三步', '四步', '五步']
            step_2 = [1, 2, 3, 4, 5]
            step_ind = step_1.index(step_)
            search_step_ = step_2[step_ind]
        else:
            search_step_ = 1

# 根据以上数据，模拟出结果，并找到 weight 大于等于 weight_，且小于等于 search_step_ 步的预测模拟结果
ls_forecast_chain = find_potential_nodes(ls_adv, ls_forbidden, weight_chain, df_net)
ls_forecast_co_ex = find_potential_nodes(ls_adv, ls_forbidden, weight_co_ex, df_nodes_co_ex_1)
df_forecast_chain = pd.DataFrame(ls_forecast_chain, columns=['t', 'origin', 'destination'])
df_forecast_co_ex = pd.DataFrame(ls_forecast_co_ex, columns=['t', 'origin', 'destination'])
df_result_chain = df_forecast_chain.loc[df_forecast_chain['t'] <= search_step_]
df_result_co_ex = df_forecast_co_ex.loc[df_forecast_co_ex['t'] <= search_step_]
ls_pre_chain = df_result_chain['destination'].unique()
ls_pre_co_ex = df_result_co_ex['destination'].unique()
ls_pre = [x for x in ls_pre_chain if x in ls_pre_co_ex]  # 共现求得的模拟结果和上下游求得的模拟结果取交集
ls_pre = [x for x in ls_pre if x not in ls_local_best and x not in ls_forbidden]
ls_pre = list(set(ls_pre))

if env_ == '补短板' or env_ == '无':
    ls_final = [x for x in ls_pre if x in ls_chain_nodes]
elif env_ == '促转型':
    ls_final = [x for x in ls_pre if x not in ls_chain_nodes]
elif env_ == '自定义' or env_ == '建集群' or env_ == '锻长板':
    ls_final = [x for x in ls_pre]

# 根据模拟场景确定全联通和算法
if env_ == '补短板' or env_ == '促转型' or env_ == '自定义':
    # 输入是否全联通
    if_connection_ = input('请输入模拟结果是否全联通（是/否）：')
    # 当选择模拟结果全联通时，需要选择具体的全联通算法：最短路径（最少的步数）/最小代价（每一步都是关联最紧密的路径）
    if if_connection_ == '是':
        path_calculation = input('请输入算法选择（最短路径/最小代价）：')
    else:
        path_calculation = '无'
elif env_ == '锻长板':
    # 用户选择是否要按照默认情况进行全联通
    if_define_connection = input('是否使用默认全联通模拟参数（是/否）:')
    if if_define_connection == '否':
        if_connection_ = input('请输入模拟结果是否全联通（是/否）：')
    else:
        if_connection_ = '是'

    # 当选择模拟结果全联通时，需要选择具体的全联通算法：最短路径（最少的步数）/最小代价（每一步都是关联最紧密的路径）
    if if_connection_ == '是':
        path_calculation = input('请输入算法选择（最短路径/最小代价）：')
    else:
        path_calculation = '无'
elif env_ == '建集群':
    # 用户选择是否要按照默认情况进行全联通
    if_define_connection = input('是否使用默认全联通模拟参数（是/否）:')
    if if_define_connection == '否':
        if_connection_ = input('请输入模拟结果是否全联通（是/否）：')
    else:
        if_connection_ = '否'

    # 当选择模拟结果全联通时，需要选择具体的全联通算法：最短路径（最少的步数）/最小代价（每一步都是关联最紧密的路径）
    if if_connection_ == '是':
        path_calculation = input('请输入算法选择（最短路径/最小代价）：')
    else:
        path_calculation = '无'

# 根据是否全联通参数，确定是否需要进行全联通判断
if if_connection_ == '是':
    # 将模拟出的节点加入到之前设置的种子产品和优势产品中
    ls_chain_local_nodes_ = ls_adv + ls_final
    # 用得到的节点编码列表（模拟条件优势产品+种子产品+模拟结果产品）构成邻接矩阵，以此计算可达矩阵，判断这些点是否全连通
    adj_matrix_chain, member_dict_chain = adjacence_matrix(ls_chain_local_nodes_, df_net)
    adj_matrix_co_ex, member_dict_co_ex = adjacence_matrix(ls_chain_local_nodes_, df_nodes_co_ex_1)
    # print(adj_matrix)
    reach_matrix_chain = get_reach_matrix(adj_matrix_chain)  # 上下游到达矩阵
    reach_matrix_co_ex = get_reach_matrix(adj_matrix_co_ex)  # 共现到达矩阵

    # 当上下游或共现的可达矩阵中，有一个值为0的元素个数为0，则说明这些节点构成的网络已经达到全连通。
    if len(np.where(reach_matrix_chain == 0)[0]) == 0 or len(np.where(reach_matrix_co_ex == 0)[0]) == 0:
        print('模拟结果已构成全联通网络，直接输出结果。')

    else:
        ls_zero_chain = {}
        for i in range(len(reach_matrix_chain)):
            ls_i = reach_matrix_chain[i].tolist()[0]
            ls_zero_chain[i] = len([x for x in ls_i if x == 0])
        min_index_chain = []
        for key, item in ls_zero_chain.items():
            if item == min(ls_zero_chain.values()):
                min_index_chain.append(key)
            # print(key, item)

        ls_zero_co_ex = {}
        for i in range(len(reach_matrix_co_ex)):
            ls_i = reach_matrix_co_ex[i].tolist()[0]
            ls_zero_co_ex[i] = len([x for x in ls_i if x == 0])
        min_index_co_ex = []
        for key, item in ls_zero_co_ex.items():
            if item == min(ls_zero_co_ex.values()):
                min_index_co_ex.append(key)

        if min(ls_zero_co_ex.values()) < min(ls_zero_chain.values()):
            reach_matrix = reach_matrix_co_ex
            min_index_z = min_index_co_ex
            member_dict = member_dict_co_ex
        else:
            reach_matrix = reach_matrix_chain
            min_index_z = min_index_chain
            member_dict = member_dict_chain

        # 当预测出的产品与模拟条件产品并没有构成全连通时，需要通过算法寻找补充节点，find_shortest_path_算法慢的不行
        if path_calculation == '最短路径':
            # 首先找到没有形成全联通的点对有哪些
            location = np.where(reach_matrix == 0)
            path_total = []
            path_weight_total = []
            ls_add = []
            for i in range(len(location[0])):
                start_index = location[0][i]
                end_index = location[1][i]
                if end_index > start_index == min_index_z[0]:
                    start_node = list(member_dict.keys())[start_index]
                    end_node = list(member_dict.keys())[end_index]
                    # 用最短路径算法找到对应的补充路径
                    path, path_weight = find_shortest_path_(start_node, end_node, all_nodes, ls_forbidden, df_net)
                    path_total.extend(path)
                    path_weight_total.append(path_weight)

                    ls_nodes = [y for x in path_total for y in x]
                    ls_nodes = list(Counter(ls_nodes))
                    ls_add_i = [x for x in ls_nodes if x not in ls_adv and x not in ls_final]
                    ls_add.extend(ls_add_i)
                    ls_add = list(Counter(ls_add))
                    ls_nodes = ls_final + ls_adv + ls_add
                    adj_matrix, member_dict_i = adjacence_matrix(ls_nodes, df_net)
                    reach_matrix = get_reach_matrix(adj_matrix)
                    if len(np.where(reach_matrix == 0)[0]) == 0:
                        # 当已经构成全联通时跳出循环得到最终结果，新增的节点ls_add用于构成全联通
                        print('已通过补充节点获得全联通')
                        break
                    # else:
                    #     print(start_index, end_index)
                    #     print(start_node, end_node)
                    #     print(len(np.where(reach_matrix == 0)[0]))
                    #     print(ls_add)
            ls_add = list(Counter(ls_add))

        elif path_calculation == '最小代价':
            # 首先找到没有形成全联通的点对有哪些
            location = np.where(reach_matrix == 0)
            path_total = []
            path_weight_total = []
            ls_add = []
            for i in range(len(location[0])):
                start_index = location[0][i]
                end_index = location[1][i]
                if end_index > start_index == min_index_z[0]:
                    start_node = list(member_dict.keys())[start_index]
                    end_node = list(member_dict.keys())[end_index]
                    # 用最小代价算法寻找全联通补充节点
                    path, path_weight = find_minimal_path_(start_node, end_node, all_nodes, ls_forbidden, df_net)
                    path_total.extend(path)
                    path_weight_total.append(path_weight)

                    ls_nodes = [y for x in path_total for y in x]
                    ls_nodes = list(Counter(ls_nodes))
                    ls_add_i = [x for x in ls_nodes if x not in ls_adv and x not in ls_final]
                    ls_add.extend(ls_add_i)
                    ls_add = list(Counter(ls_add))
                    ls_nodes = ls_final + ls_adv + ls_add
                    adj_matrix, member_dict_i = adjacence_matrix(ls_nodes, df_net)
                    reach_matrix = get_reach_matrix(adj_matrix)
                    if len(np.where(reach_matrix == 0)[0]) == 0:
                        # 当已经构成全联通时跳出循环得到最终结果，新增的节点ls_add用于构成全联通
                        print('已通过补充节点获得全联通')
                        break
                    # else:
                    #     print(start_index, end_index)
                    #     print(start_node, end_node)
                    #     print(len(np.where(reach_matrix == 0)[0]))
                    #     print(ls_add)
            ls_add = list(Counter(ls_add))

        # 得到补充节点后，针对补充节点进行评估，与正式模拟出的节点评估分开
        ls_final_add = []
        for k in ls_add:
            ls_i = [k, code_to_name(k, df_nodes), code_to_chain(k, df_nodes)]
            t_k = 1
            # 然后找到该节点几步以内有多少本地节点
            ls_connect_k = find_connect_nodes(k, t_k, df_net)
            df_k_local = df_nodes.loc[(df_nodes['Id'].isin(ls_connect_k)) & (df_nodes['local'] == '1')]
            ls_i.append(len(df_k_local))

            # 看这些本地节点分别属于哪条产业链
            ls_chain_ = []
            ls_chain_k = df_k_local['chain'].tolist()
            for b in ls_chain_k:
                if len(b) > 1:
                    # print(b)
                    chain = json.loads(b.replace("'", '"'))
                    ls_chain_.extend(chain)
            ls_i.append(list(set(ls_chain_)))
            ls_final_add.append(ls_i)

        # 打印查看补充产品节点结果
        df_final_add = pd.DataFrame(ls_final_add, columns=['product_code',
                                                           'product_name', 'chain', 'local_support', 'loc_sup_chain'])
        df_final_add.sort_values(['local_support'], ascending=False, inplace=True)
        print('补充结果评价', len(df_final_add), df_final_add)

elif if_connection_ == '否' and path_calculation == '无':
    print('模拟结果不需要全联通，直接输出结果。')

# 当已经构成全连通网络，则不需要新增节点，与全联通参数选否的情况一样，直接对当前模拟出来的节点按照本地支持力度进行排序
# 根据预测到的节点编码，补充该节点属于哪条或哪几条产业链，以及其本地关联产品属于哪些产业链
ls_final_chain = []
for k in ls_final:
    ls_i = [k, code_to_name(k, df_nodes), code_to_chain(k, df_nodes)]
    t_k = 1
    # 然后找到该节点几步以内有多少本地节点
    ls_connect_k = find_connect_nodes(k, t_k, df_net)
    df_k_local = df_nodes.loc[(df_nodes['Id'].isin(ls_connect_k)) & (df_nodes['local'] == '1')]
    ls_i.append(len(df_k_local))

    # 看这些本地节点分别属于哪条产业链
    ls_chain_ = []
    ls_chain_k = df_k_local['chain'].tolist()
    for b in ls_chain_k:
        if len(b) > 1:
            # print(b)
            chain = json.loads(b.replace("'", '"'))
            ls_chain_.extend(chain)
    ls_i.append(list(set(ls_chain_)))
    ls_final_chain.append(ls_i)

# 打印查看模拟出的属于该产业链的产品节点结果
df_final = pd.DataFrame(ls_final_chain,
                        columns=['product_code', 'product_name', 'chain', 'local_support', 'loc_sup_chain'])
df_final.sort_values(['local_support'], ascending=False, inplace=True)

print('模拟结果评价', len(df_final),df_final)

请输入模拟模式（快速模拟/高级模拟）：高级模拟
请输入模拟场景(补短板/锻长板/促转型/建集群/自定义)：锻长板
请输入产业链名称：["风电"]
是否选择预置的优势产品（是/否）：是
请输入要参与模拟的优势产品：[]
请输入不参与模拟的优势产品：[]
请输入参与模拟的种子产品：[]
是否使用预置的排除产品（是/否）：是
请输入要增加的排除产品：[]
请输入要减少的排除产品：[]
是否使用默认搜索阈值（是/否）:是
请输入模拟的搜索半径（一步/两步/三步/四步/五步）：两步
是否使用默认全联通模拟参数（是/否）:是
请输入算法选择（最短路径/最小代价）：最小代价
已通过补充节点获得全联通
补充结果评价 39    product_code   product_name                   chain  local_support                                                loc_sup_chain
0        760907       机械工程设计服务                       0             47             [榆神特色可降解塑料产业链, 风电产业链, 能源化工装备制造产业链, 智能无人系统和通航产业链]
37      3516110         金属轧制设备                       0             33                        [铝及铝合金产业链, 风电产业链, 能源化工装备制造产业链, 光伏产业链]
3       1800010             服装   [榆神特色可降解塑料产业链, 轻纺产业链]             23                                               [轻纺产业链, 风电产业链]
9    2502990000         其他石油制品                       0             20  [风电产业链, 兰炭-金属镁产业链, 大宗固废产业链, 氢能产业链, 通用工程塑料产业链, 榆神特色可降解塑料产业链]
11    701000000             原油        

In [57]:
full_path_(df_forecast_chain,'3111020205')

[['1011010', '3111020205']]

In [87]:
len(ls_pre)

168

In [4]:
ls_adv

['3130710',
 '2651010',
 '1011010',
 '3462010',
 '2641010',
 '2659010',
 '2820010',
 '2502070000',
 '310905',
 '3051030',
 '2651040',
 '74']

In [82]:
ls_adv

['2651010',
 '2502070000',
 '3462010',
 '1011010',
 '310905',
 '74',
 '2659010',
 '3130710',
 '2820010',
 '3051030',
 '2651040',
 '2641010']

In [88]:
df_result_chain

,t,origin,destination
0,1,2651010,2602
1,1,2651010,260202
2,1,2651010,260209
3,1,2651010,260213
4,1,2651010,3917
5,1,2651010,63190213
6,1,2651010,63190214
7,1,2651010,365502
8,1,2651010,2408
9,1,2651010,2406


In [89]:
df_result_co_ex

,t,origin,destination
0,1,2651010,2608040300
1,1,2651010,260802
2,1,2651010,26080304
3,1,2651010,26
4,1,2651010,2608040200
...,...,...,...
22339,2,3507040000,3538010301
22340,2,3507040000,3538010302
22341,2,3507040000,353804
22342,2,3507040000,35380103


In [50]:
ls_chain_local_nodes

['3130710',
 '2651010',
 '1011010',
 '3462010',
 '2641010',
 '2659010',
 '2820010',
 '2502070000',
 '310905',
 '3051030',
 '2651040',
 '74']

In [49]:
ls_local

['3982710',
 '3572020',
 '2620020',
 '3973710',
 '3973730',
 '3973731',
 '3973732',
 '3532050',
 '3531010',
 '3467710',
 '3981710',
 '3130710',
 '119',
 '3130712',
 '3130738',
 '3130736',
 '3130711',
 '3130739',
 '3130737',
 '3130713',
 '2911011',
 '2911013',
 '3130715',
 '3130714',
 '3516110',
 '3974760',
 '1371010',
 '3972720',
 '2921020',
 '1310010',
 '1350010',
 '1350020',
 '1340010',
 '3821020',
 '3823050',
 '3823030',
 '3821080',
 '1713010',
 '2502990000',
 '3453720',
 '2501000000',
 '3532040300',
 '701000000',
 '1522010',
 '1494010',
 '1310030',
 '3214010',
 '1019010',
 '3130724',
 '3130729',
 '3130727',
 '130501',
 '1331020',
 '2612010',
 '3812000',
 '2651010',
 '1361010',
 '3251710',
 '3812010',
 '3812020',
 '1321720',
 '261413',
 '30102',
 '3216010',
 '3911010',
 '3911040',
 '3911030',
 '3911020',
 '3911050',
 '3911060',
 '3973720',
 '3981720',
 '301',
 '2611010',
 '3054010',
 '3252790',
 '3055010',
 '3056010',
 '3841060',
 '702010000',
 '702020100',
 '703100000',
 '702020000

In [48]:
ls_pre

['3431010101',
 '3431010199',
 '3618010211',
 '260202',
 '3653020200',
 '63190102',
 '3431010103',
 '3459010',
 '350701',
 '3577010',
 '3041010',
 '260209',
 '3431',
 '3111',
 '2501000000',
 '2602',
 '26130201',
 '26021501',
 '3431010104',
 '3431010102',
 '3528040000',
 '343101',
 '3501',
 '260213']

In [39]:
ls_pre_co_ex

['6699',
 '1',
 '380105',
 '4212010000',
 '4212',
 '2100010',
 '4212030300',
 '4212030200',
 '2101060101',
 '4212030100',
 '421203',
 '4212040400',
 '21010602',
 '4212039900',
 '4212040100',
 '3801050500',
 '3801050700',
 '2101080201',
 '2101070202',
 '2101079902',
 '2101070103',
 '2101070201',
 '2101080202',
 '2101079903',
 '2101070204',
 '2101070205',
 '2101070101',
 '2101079901',
 '2101070203',
 '2190010',
 '21010802',
 '42',
 '3033710',
 '3109',
 '3031070',
 '3031010',
 '1019010',
 '1004030102',
 '10040102',
 '100403',
 '65',
 '6502',
 '3411020',
 '3411010',
 '3501060100',
 '3501',
 '35',
 '3501040203',
 '3501060202',
 '8405020400',
 '8405010000',
 '8405',
 '8405020300',
 '8405021200',
 '5902',
 '1522010',
 '15',
 '1520010',
 '3311',
 '3214010',
 '3316040100',
 '3216010',
 '3316',
 '3211020',
 '333003',
 '3330030201',
 '3213010',
 '3217010',
 '3330030100',
 '3315',
 '3318',
 '333001',
 '3312',
 '331201',
 '3212020',
 '130402',
 '3212030',
 '1304',
 '302',
 '30201',
 '331102',
 '302

In [38]:
ls_pre_chain

['801',
 '810010',
 '810020',
 '3516110',
 '3140740',
 '350701',
 '3404',
 '2406',
 '3525010',
 '240705',
 '3853050',
 '3653020200',
 '3624',
 '3539',
 '3418010100',
 '3459010',
 '3431010101',
 '3431010102',
 '3431010103',
 '3431010104',
 '3431010199',
 '3431',
 '343101',
 '3444040',
 '3501',
 '3515010',
 '3604010800',
 '3604019900',
 '351702',
 '3534010',
 '363902',
 '3621',
 '3577010',
 '363102',
 '2602',
 '260202',
 '260209',
 '260213',
 '3917',
 '63190213',
 '63190214',
 '365502',
 '2408',
 '360402',
 '2612030',
 '3011010',
 '3111',
 '3111020205',
 '3528040000',
 '35300401',
 '3604090200',
 '3604090400',
 '3608120100',
 '3610030100',
 '3618010211',
 '3130030303',
 '26021501',
 '1800010',
 '701000000',
 '2501000000',
 '2502990000',
 '63190102',
 '250402',
 '26130201',
 '3604070400',
 '2521020000',
 '3041010',
 '360403',
 '360501',
 '2503030400',
 '7702010000',
 '3140710',
 '320901',
 '7801010100',
 '7801010300',
 '5601010000',
 '2502000000',
 '250212',
 '25021202',
 '2502120201',
 '

In [19]:
df_final

,product_code,product_name,chain,local_support,loc_sup_chain
6,3111,玻璃,"[通用工程塑料产业链, 光伏产业链]",6,"[风电产业链, 通用工程塑料产业链, 光伏产业链]"
0,3209020111,金属硅,[光伏产业链],3,[光伏产业链]
1,2618090400,多晶硅片,[光伏产业链],2,[光伏产业链]
2,3001011703,乙烯-乙酸乙烯酯（EVA）塑料农用薄膜,[光伏产业链],2,"[乙烯产业链, 光伏产业链]"
3,2613010407,聚氟乙烯,[光伏产业链],1,"[乙烯产业链, 光伏产业链]"
4,3913060499,其他太阳能电池零部件,[光伏产业链],1,"[乙烯产业链, 光伏产业链]"
5,30010106,聚酯塑料薄膜,[光伏产业链],1,[光伏产业链]
7,3001011300,聚四氟乙烯薄膜,[光伏产业链],1,[光伏产业链]
8,3913060401,太阳能电池板,[光伏产业链],1,[光伏产业链]


In [26]:
ls1=df_final['product_code'].unique().tolist()
ls2=df_final['local_support'].unique().tolist()
[x for x in (ls1 +ls2)]

['3111',
 '3209020111',
 '2618090400',
 '3001011703',
 '2613010407',
 '3913060499',
 '30010106',
 '3001011300',
 '3913060401',
 6,
 3,
 2,
 1]

In [ ]:
df_final['local_support'].unique()

In [18]:
df_nodes_co_ex_1 = pd.read_csv('E:\\yulin\\connection\\co_exist_tupu\\nodes_weighted23f3_75_pp_2022_1_mst_thre_clear.csv', dtype={'weight':'float','source':'str','target':'str'})
df_nodes_co_ex_1

,source,target,weight,ls
0,401990100,130999,0.808511,"130999,401990100"
1,401990100,1309999900,0.808511,"1309999900,401990100"
2,401990100,1309060700,0.808511,"1309060700,401990100"
3,401050200,130999,0.808511,"130999,401050200"
4,401050200,1309999900,0.808511,"1309999900,401050200"
...,...,...,...,...
85419,3501060202,7499,0.656250,"3501060202,7499"
85420,65,9303050200,0.439394,"65,9303050200"
85421,6502070300,9303050200,0.439394,"6502070300,9303050200"
85422,6502070200,9303050200,0.439394,"6502070200,9303050200"


In [16]:
df_nodes_co_ex_1 = pd.read_csv('E:\\yulin\\connection\\co_exist_tupu\\nodes_weighted23f3_75_pp_2022_1_mst_thre_clear.csv', dtype={'weight':'float','source':'str','target':'str'})
df_nodes_co_ex_1

,source,target,weight
0,401990100,130999,0.808511
1,401990100,1309999900,0.808511
2,401990100,1309060700,0.808511
3,401050200,130999,0.808511
4,401050200,1309999900,0.808511
...,...,...,...
91626,6901030000,7499,0.657143
91627,65,9303050200,0.439394
91628,6502070300,9303050200,0.439394
91629,6502070200,9303050200,0.439394


In [12]:
df_nodes_co_ex_1['ls'] = list(map(lambda x,y: [x, y], df_nodes_co_ex_1['source'], df_nodes_co_ex_1['target']))
df_nodes_co_ex_1['ls'] = df_nodes_co_ex_1['ls'].apply(lambda x:','.join(sorted(x)))
df_nodes_co_ex_1.drop_duplicates(subset=['ls'],keep = 'first')

,source,target,weight,ls
0,401990100,130999,0.808511,"130999,401990100"
1,401990100,1309999900,0.808511,"1309999900,401990100"
2,401990100,1309060700,0.808511,"1309060700,401990100"
3,401050200,130999,0.808511,"130999,401050200"
4,401050200,1309999900,0.808511,"1309999900,401050200"
...,...,...,...,...
91622,3501060202,7499,0.656250,"3501060202,7499"
91627,65,9303050200,0.439394,"65,9303050200"
91628,6502070300,9303050200,0.439394,"6502070300,9303050200"
91629,6502070200,9303050200,0.439394,"6502070200,9303050200"


In [17]:
df_nodes_co_ex_2 = df_nodes_co_ex_1.drop_duplicates(subset=['source','target'])
df_nodes_co_ex_2['ls'] = list(map(lambda x,y: [x, y], df_nodes_co_ex_2['source'], df_nodes_co_ex_2['target']))
df_nodes_co_ex_2['ls'] = df_nodes_co_ex_2['ls'].apply(lambda x:','.join(sorted(x)))
df_nodes_co_ex_2.drop_duplicates(subset=['ls'],keep = 'first').to_csv('E:\\yulin\\connection\\co_exist_tupu\\nodes_weighted23f3_75_pp_2022_1_mst_thre_clear.csv',index = False,mode = 'w')

<ipython-input-17-f97013914ff2>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_nodes_co_ex_2['ls'] = list(map(lambda x,y: [x, y], df_nodes_co_ex_2['source'], df_nodes_co_ex_2['target']))
<ipython-input-17-f97013914ff2>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_nodes_co_ex_2['ls'] = df_nodes_co_ex_2['ls'].apply(lambda x:','.join(sorted(x)))


In [14]:
df_nodes_co_ex_2 = df_nodes_co_ex_1.drop_duplicates()
df_nodes_co_ex_2.drop_duplicates(subset=['source','target'])

,source,target,weight,ls
0,401990100,130999,0.808511,"130999,401990100"
1,401990100,1309999900,0.808511,"1309999900,401990100"
2,401990100,1309060700,0.808511,"1309060700,401990100"
3,401050200,130999,0.808511,"130999,401050200"
4,401050200,1309999900,0.808511,"1309999900,401050200"
...,...,...,...,...
91622,3501060202,7499,0.656250,"3501060202,7499"
91627,65,9303050200,0.439394,"65,9303050200"
91628,6502070300,9303050200,0.439394,"6502070300,9303050200"
91629,6502070200,9303050200,0.439394,"6502070200,9303050200"


In [ ]:
df_nodes_co_ex_2.to_csv('E:\\yulin\\connection\\co_exist_tupu\\nodes_weighted23f3_75_pp_2022_1_mst_thre_clear.csv',index = False,mode = 'w')

In [6]:
def find_potential_nodes(ls_adv, ls_local, ls_forbidden, wight_value, df_product_net):
    # ls_adv是本地优势产品，list
    # ls_local是本地所有产品列表，list
    # forbidden是预先设置的排除产业，list
    # wight_value是设定的阈值，float
    # df_product_net是产品上下游联系网络，dataframe

    _district_industryCOs = ls_adv
    industryCOs_product = ls_local
    ls_forbidden = ls_forbidden
    _simulate_list = []
    _simulate_list.append(_district_industryCOs)
    # 从一步开始向多步扩展
    flag = 1
    t = 1
    ls_forcast = []
    while flag == 1:
        _exist_list = []
        # 把所有优势产品输入_exist_list
        for k in range(0, len(_simulate_list)):
            _exist_list.extend(_simulate_list[k])
        c = 0
        _possible_list = []

        for j in range(len(_simulate_list[-1])):
            _industryCO = _simulate_list[-1][j]
            if _industryCO in industryCOs_product and _industryCO not in ls_forbidden:
                # 判断该优势产品属于本地产品，且不属于本地排除产品，然后从该优势产品开始向外找，一步联系值大于等于wight_value的节点
                _df_industryCO_proximity = df_product_net.loc[((df_product_net['source'] == _industryCO) |
                                                               (df_product_net['target'] == _industryCO)) &
                                                              (df_product_net['weight'] > wight_value)]
                if len(_df_industryCO_proximity) > 0:
                    # 取出该一步联系所能联系到的所有产品节点
                    _industryCOs_1 = _df_industryCO_proximity.loc[:, 'source'].unique()
                    _industryCOs_2 = _df_industryCO_proximity.loc[:, 'target'].unique()

                    for m in range(len(_industryCOs_1)):
                        _industryCO_ = _industryCOs_1[m]
                        # 如果该产品节点不是已经找过的产品（作为起点的产品），也不属于本地排除产品，则加入到_possible_list
                        if _industryCO_ not in _exist_list and _industryCO not in ls_forbidden:
                            _possible_list.append(_industryCO_)
                            ls_a = []
                            ls_a.append(t)
                            ls_a.append(_industryCO)
                            ls_a.append(_industryCO_)

                            ls_forcast.append(ls_a)
                            c = c + 1

                    for m in range(len(_industryCOs_2)):
                        _industryCO_ = _industryCOs_2[m]
                        # 如果该产品节点不是已经找过的产品（作为起点的产品），则加入到_possible_list
                        if _industryCO_ not in _exist_list and _industryCO not in ls_forbidden:
                            _possible_list.append(_industryCO_)
                            ls_a = []
                            ls_a.append(t)
                            ls_a.append(_industryCO)
                            ls_a.append(_industryCO_)
                            ls_forcast.append(ls_a)
                            c = c + 1
        if c == 0:
            # 当c等于0时，说明一步联系找到的都是已有产品，无法再继续找了
            flag = 0
        else:
            # 当c不等于0时，说明还可以继续往下找
            flag = 1
            t = t + 1

            _possible_list = list(set(_possible_list))
            _simulate_list.append(_possible_list)
    return ls_forcast

In [7]:
def find_potential_nodes_(ls_adv, ls_forbidden, wight_value, df_product_net):
    # ls_adv是本地优势产品，list
    # ls_local是本地所有产品列表，list
    # forbidden是预先设置的排除产业，list
    # wight_value是设定的阈值，float
    # df_product_net是产品上下游联系网络，dataframe

    _district_industryCOs = ls_adv
    ls_forbidden = ls_forbidden
    _simulate_list = [_district_industryCOs]
    # 从一步开始向多步扩展
    flag = 1
    t = 1
    ls_forcast = []
    while flag == 1:
        _exist_list = []
        # 把所有优势产品输入_exist_list
        for k in range(0, len(_simulate_list)):
            _exist_list.extend(_simulate_list[k])
        c = 0
        _possible_list = []

        for j in range(len(_simulate_list[-1])):
            _industryCO = _simulate_list[-1][j]
            if _industryCO not in ls_forbidden:
                # 判断该优势产品属于本地产品，且不属于本地排除产品，然后从该优势产品开始向外找，一步联系值大于等于wight_value的节点
                _df_industryCO_proximity = df_product_net.loc[((df_product_net['source'] == _industryCO) |
                                                               (df_product_net['target'] == _industryCO)) &
                                                              (df_product_net['weight'] >= wight_value)]
                if len(_df_industryCO_proximity) > 0:
                    # 取出该一步联系所能联系到的所有产品节点
                    _industryCOs_1 = _df_industryCO_proximity.loc[:, 'source'].unique()
                    _industryCOs_2 = _df_industryCO_proximity.loc[:, 'target'].unique()

                    for m in range(len(_industryCOs_1)):
                        _industryCO_ = _industryCOs_1[m]
                        # 如果该产品节点不是已经找过的产品（作为起点的产品），也不属于本地排除产品，则加入到_possible_list
                        if _industryCO_ not in _exist_list and _industryCO not in ls_forbidden:
                            _possible_list.append(_industryCO_)
                            ls_a = []
                            ls_a.append(t)
                            ls_a.append(_industryCO)
                            ls_a.append(_industryCO_)

                            ls_forcast.append(ls_a)
                            c = c + 1

                    for m in range(len(_industryCOs_2)):
                        _industryCO_ = _industryCOs_2[m]
                        # 如果该产品节点不是已经找过的产品（作为起点的产品），则加入到_possible_list
                        if _industryCO_ not in _exist_list and _industryCO not in ls_forbidden:
                            _possible_list.append(_industryCO_)
                            ls_a = []
                            ls_a.append(t)
                            ls_a.append(_industryCO)
                            ls_a.append(_industryCO_)
                            ls_forcast.append(ls_a)
                            c = c + 1
        if c == 0:
            # 当c等于0时，说明一步联系找到的都是已有产品，无法再继续找了
            flag = 0
        else:
            # 当c不等于0时，说明还可以继续往下找
            flag = 1
            t = t + 1

            _possible_list = list(set(_possible_list))
            _simulate_list.append(_possible_list)
    return ls_forcast

In [68]:
ls_connection_weight = ['最强', '较强', '中等', '较弱', '最弱']
ls_connection_weight_chain = [4.5, 3.5, 2.5, 1.5, 0.5]
ls_connection_weight_co_ex = [0.8, 0.6, 0.4, 0.2, 0.1]

chain = input('请输入产业链名称：')
ls_chain = json.loads(chain)

# 找到模拟对象产业链（可能不止一条产业链）中所有本地节点
df_local_chain = pd.DataFrame([])
for n in ls_chain:
    df_chain_local_n = df_nodes.loc[(df_nodes['chain'].str.contains(n)) & (df_nodes['local'] == '1')]
    df_local_chain = pd.concat([df_local_chain, df_chain_local_n], axis=0)

df_local_chain = df_local_chain.drop_duplicates(inplace=False)
ls_chain_local_nodes = df_local_chain['Id'].tolist()

# 根据模拟产业链，得到这些产业链由哪些节点组成
ls_chain_nodes = find_chain_ls(ls_chain, df_nodes)

# 快速模拟，条件配置：
ls_adv = ls_chain_local_nodes  # 优势产业为输入的产业链所有本地产品，种子产业为无
ls_local = ls_local_best
# 排除产品名称：原铝（电解铝），铝压延加工材,铝材,铝合金,粗纱机,毛纱,羊毛纱
# 排除产品代码：'331603','3336','333601','33340105','362808','170202','17020201'
ls_forbidden = ['331603', '3336', '3316', '333601', '33340105', '362808', '170202', '17020201']
index_weight = ls_connection_weight.index('中等')
weight_chain = ls_connection_weight_chain[index_weight]  # 上下游阈值大于等于3.5(较强)
weight_co_ex = ls_connection_weight_co_ex[index_weight]  # 上下游阈值大于等于0.6(较强)

search_step_ = 1  # 步数小于等于2步
if_connection_ = '否'  # 是否全连通：********否*********
path_calculation = '无'  # 算法名称：无

请输入产业链名称：["文化旅游"]


In [63]:
ls_forecast_chain = find_potential_nodes(ls_adv, ls_local_best, ls_forbidden, weight_chain, df_net)
ls_forecast_chain_ = find_potential_nodes_(ls_adv, ls_forbidden, weight_chain, df_net)

In [64]:
# ls_forecast_chain2 = find_potential_nodes(ls_adv, ls_local_best, ls_forbidden, weight_chain, df_nodes_co_ex_1)
ls_forecast_chain_2 = find_potential_nodes_(ls_adv, ls_forbidden, weight_co_ex, df_nodes_co_ex_1)

In [69]:
ls_adv

['3979040']

In [65]:
df1= pd.DataFrame(ls_forecast_chain)
df2 = pd.DataFrame(ls_forecast_chain_)
df3 = pd.DataFrame(ls_forecast_chain_2)

In [32]:
pd.set_option('display.max_columns', 1000)
pd.set_option('display.width', 1000)
pd.set_option('display.max_colwidth', 1000)
pd.set_option("display.max_rows",1000)

In [67]:
ls1=df3.loc[df3[0] <= 1][2].unique().tolist()
ls2=df2.loc[df2[0] <= 1][2].unique().tolist()
ls_pre = [x for x in ls1 if x in ls2]  # 共现求得的模拟结果和上下游求得的模拟结果取交集
ls_pre = [x for x in ls_pre if x not in ls_local_best and x not in ls_forbidden]
ls_pre = list(set(ls_pre))
ls_pre

['365502', '350205', '4001', '3718030200']

In [58]:
ls_pre = [x for x in ls1 if x in ls2]
ls_pre

['350205', '3316', '3217010']

In [56]:
ls1

['4001060200',
 '4001',
 '400102',
 '4001030201',
 '400101',
 '4001030203',
 '4001030202',
 '400103',
 '3921080',
 '4001030299',
 '40',
 '4127990400',
 '3430',
 '3428060000',
 '34',
 '3429030100',
 '343001',
 '3431010103',
 '3459010',
 '3431010199',
 '3431010101',
 '3431010104',
 '3431',
 '3431010102',
 '343101',
 '63130102',
 '6313',
 '63',
 '412402',
 '6699',
 '1',
 '380105',
 '4212010000',
 '4212',
 '2100010',
 '4212030300',
 '4212030200',
 '2101060101',
 '4212030100',
 '421203',
 '4212040400',
 '21010602',
 '4212039900',
 '4212040100',
 '3801050500',
 '3801050700',
 '2101080201',
 '2101070202',
 '2101079902',
 '2101070103',
 '2101070201',
 '2101080202',
 '2101079903',
 '2101070204',
 '2101070205',
 '2101070101',
 '2101079901',
 '2101070203',
 '2190010',
 '21010802',
 '42',
 '3130710',
 '3130712',
 '3208993900',
 '3130738',
 '3130736',
 '3130711',
 '3130739',
 '3130737',
 '3130713',
 '3130715',
 '3130714',
 '32080234',
 '3130724',
 '3130729',
 '3130727',
 '1302',
 '301',
 '32080233'

In [59]:
df3.loc[df3[0] <= 2]

,0,1,2
0,1,3973710,4001060200
1,1,3973710,4001
2,1,3973710,400102
3,1,3973710,4001030201
4,1,3973710,400101
...,...,...,...
15654,2,1403020206,3130020202
15655,2,1403020206,3130030307
15656,2,1403020206,313003
15657,2,1403020206,31300101


In [26]:
df2.loc[df2[0] == 1]

,0,1,2
0,1,261413,250402
1,1,261413,2521020000
2,1,261413,2610
3,1,3216010,2601070200
4,1,3216010,90107
5,1,3216010,3316
6,1,3216010,4208030500
7,1,2614210,601050000
8,1,2614210,4501050000
9,1,2614210,2602160100


In [37]:
df2.loc[df2[0] == 1]

,0,1,2
0,1,261413,250402
1,1,261413,2521020000
2,1,3216010,2601070200
3,1,3216010,90107
4,1,3216010,3316
5,1,2614210,4501050000
6,1,2614210,2602160100
7,1,2614210,26021303
8,1,2614210,2614250
9,1,2614210,2602130302


In [38]:
df1.loc[df1[0] == 1]

,0,1,2
0,1,261413,250402
1,1,261413,2521020000
2,1,3216010,2601070200
3,1,3216010,90107
4,1,3216010,3316
5,1,2614210,4501050000
6,1,2614210,2602160100
7,1,2614210,26021303
8,1,2614210,2614250
9,1,2614210,2602130302
